# Change log

* ### V1:
Scheduler (CosineAnnealingWarmRestarts) step in training after every batch instead of after every epoch (see: https://pytorch.org/docs/stable/optim.html#algorithms)


* ### V2:
Using hyperparameters (learning rate, weight decay and batch size) found with RayTune


* ### V3:
Using image augmentations found with AutoAlbument


# Data Loading

In [ ]:
import os

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
os.listdir('../input/cassava-leaf-disease-classification')

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
label_map = pd.read_json('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 
                         orient='index')
display(train.head())
display(test.head())
display(label_map)

In [ ]:
sns.distplot(train['label'], kde=False)

# Directory settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

checkpoint_dir = './raytune/checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

INPUT_DIR = '../input/cassava-resnext50-32x4d-starter-training/'

# CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    apex=False
    print_freq=100
    num_workers=4
    model_name='resnext50_32x4d'
    size=256
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=10
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    #T_max=10 # CosineAnnealingLR
    T_0=10 # CosineAnnealingWarmRestarts
    #lr=1e-4
    lr=0.00017490710557883633  # RayTune
    min_lr=1e-6
    batch_size=32  # RayTune
    #weight_decay=1e-6
    weight_decay=0.00048429019694106525  # RayTune
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=False
    inference=True
    tune=False
    one_fold = True
    
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)
    
if CFG.tune:
    #CFG.n_fold = 1
    CFG.trn_fold = [0]
    
if CFG.one_fold:
    CFG.trn_fold = [0]

# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

# === Albumentations ===
#!pip install --upgrade albumentations
import albumentations as A
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations import *
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if CFG.tune:
    from functools import partial
    from ray import tune
    from ray.tune import CLIReporter
    from ray.tune.schedulers import ASHAScheduler

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# CV split

In [ ]:
folds = train.copy()
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_col])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', CFG.target_col]).size())

# Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        return image, label
    

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
train_dataset = TrainDataset(train, transform=None)

for i in range(1):
    image, label = train_dataset[i]
    plt.imshow(image)
    plt.title(f'label: {label}')
    plt.show() 

# Transforms

In [ ]:
train_aug_policy = Compose([
  RandomResizedCrop(CFG.size, CFG.size),
  OneOf([
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.0577419615907971e-10, r_shift_limit=(222, 222), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.1570925251011518e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(212, 212), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.001060941919217441, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-93, -93)),
        RandomBrightnessContrast(always_apply=False, p=0.0015757614107849705, brightness_limit=(-0.3257349729537964, -0.3257349729537964), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.501223852901574e-11, brightness_limit=(0, 0), contrast_limit=(2.0700290203094482, 2.0700290203094482), brightness_by_max=True),
        Solarize(always_apply=False, p=5.4632774159726655e-09, threshold=(58, 58)),
        HorizontalFlip(always_apply=False, p=0.4736693111709531),
        VerticalFlip(always_apply=False, p=9.48706013704055e-09),
        ShiftScaleRotate(always_apply=False, p=4.731186855631326e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-170.95867919921875, -170.95867919921875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.6198008331173974e-05, shift_limit_x=(0.1533520221710205, 0.1533520221710205), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0003928436627583104, shift_limit_x=(0, 0), shift_limit_y=(0.3387199640274048, 0.3387199640274048), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.219075934491662e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.878372669219971, 5.878372669219971), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.8317598431413325e-09, max_holes=16, max_height=303, max_width=303, min_holes=16, min_height=303, min_width=303, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.082051537567148e-07, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5232422803529947),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.160410765370216e-09, r_shift_limit=(-177, -177), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.544280620566022e-11, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-200, -200), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.397768912936066e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-208, -208)),
        RandomBrightnessContrast(always_apply=False, p=6.675312747562122e-10, brightness_limit=(0.9055547714233398, 0.9055547714233398), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(0, 0), contrast_limit=(6.213574409484863, 6.213574409484863), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0001598028158322374, threshold=(140, 140)),
        HorizontalFlip(always_apply=False, p=0.8004905952386565),
        VerticalFlip(always_apply=False, p=4.192219476361486e-06),
        ShiftScaleRotate(always_apply=False, p=3.914884686689619e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-156.5478973388672, -156.5478973388672), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0182478370374933e-11, shift_limit_x=(0.35631275177001953, 0.35631275177001953), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.329022533690511e-09, shift_limit_x=(0, 0), shift_limit_y=(0.3451913595199585, 0.3451913595199585), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4366754067696947e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.023794174194336, 9.023794174194336), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.8326412536095315e-12, max_holes=16, max_height=517, max_width=517, min_holes=16, min_height=517, min_width=517, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.0341719487049253e-10, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.19934004725832488),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.571554502522269e-09, r_shift_limit=(133, 133), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0004248523551047684, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-150, -150), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.0113839717698766e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(54, 54)),
        RandomBrightnessContrast(always_apply=False, p=1.4805081376798327e-09, brightness_limit=(-0.7113110423088074, -0.7113110423088074), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.802821381502168e-10, brightness_limit=(0, 0), contrast_limit=(5.825443744659424, 5.825443744659424), brightness_by_max=True),
        Solarize(always_apply=False, p=7.96941753101283e-10, threshold=(235, 235)),
        HorizontalFlip(always_apply=False, p=0.9334534159625107),
        VerticalFlip(always_apply=False, p=7.949419747702025e-06),
        ShiftScaleRotate(always_apply=False, p=4.1757057275356607e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-0.9680023193359375, -0.9680023193359375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00016256345533732786, shift_limit_x=(0.6186466217041016, 0.6186466217041016), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2013752104711244e-10, shift_limit_x=(0, 0), shift_limit_y=(-1.0, -1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.903889473802143e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(10.0, 10.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.7520826830124055e-10, max_holes=16, max_height=154, max_width=154, min_holes=16, min_height=154, min_width=154, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.3082903573082817e-06, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.06593778230694547),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.1207720496017458e-07, r_shift_limit=(246, 246), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9850038881487886e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(98, 98), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.383350945794493e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-75, -75)),
        RandomBrightnessContrast(always_apply=False, p=2.839564374779662e-06, brightness_limit=(0.7906588315963745, 0.7906588315963745), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.683701113583373e-10, brightness_limit=(0, 0), contrast_limit=(6.8198466300964355, 6.8198466300964355), brightness_by_max=True),
        Solarize(always_apply=False, p=1.3899642213439131e-09, threshold=(58, 58)),
        HorizontalFlip(always_apply=False, p=3.5832828378179005e-07),
        VerticalFlip(always_apply=False, p=0.0565037524845049),
        ShiftScaleRotate(always_apply=False, p=5.270709751589405e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-59.18565368652344, -59.18565368652344), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6771133834361622e-05, shift_limit_x=(0.004806637763977051, 0.004806637763977051), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.5949913032197225e-06, shift_limit_x=(0, 0), shift_limit_y=(0.9012582302093506, 0.9012582302093506), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4413740556772818e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.99999993922529e-09, 9.99999993922529e-09), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.021591672594281e-06, max_holes=16, max_height=106, max_width=106, min_holes=16, min_height=106, min_width=106, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=6.969547587290868e-08, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9434450113905214),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.1094998011528358e-05, r_shift_limit=(-130, -130), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.6268309720387206e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-144, -144), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.0838105204878137e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(72, 72)),
        RandomBrightnessContrast(always_apply=False, p=2.9960828981598364e-07, brightness_limit=(-0.22258180379867554, -0.22258180379867554), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.03912220527468868, brightness_limit=(0, 0), contrast_limit=(1.0401663780212402, 1.0401663780212402), brightness_by_max=True),
        Solarize(always_apply=False, p=2.197128310390756e-11, threshold=(197, 197)),
        HorizontalFlip(always_apply=False, p=1.323273146086266e-08),
        VerticalFlip(always_apply=False, p=1.975645093162994e-07),
        ShiftScaleRotate(always_apply=False, p=2.1148673524743486e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-8.852371215820312, -8.852371215820312), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.533228289566599e-05, shift_limit_x=(0.3565671443939209, 0.3565671443939209), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.5827334609358984e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.316303551197052, -0.316303551197052), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.9017236125743163e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.262063980102539, 7.262063980102539), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.3124598445362537e-08, max_holes=16, max_height=110, max_width=110, min_holes=16, min_height=110, min_width=110, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.9282267744269812e-05, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9608094125621085),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=9.027527669388863e-06, r_shift_limit=(53, 53), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-3, -3), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.2635405691083559, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(82, 82)),
        RandomBrightnessContrast(always_apply=False, p=4.067135227263613e-05, brightness_limit=(0.12148475646972656, 0.12148475646972656), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.581164537082792e-05, brightness_limit=(0, 0), contrast_limit=(6.851958274841309, 6.851958274841309), brightness_by_max=True),
        Solarize(always_apply=False, p=0.011123813999053045, threshold=(136, 136)),
        HorizontalFlip(always_apply=False, p=2.010093685499255e-08),
        VerticalFlip(always_apply=False, p=4.426268159511741e-06),
        ShiftScaleRotate(always_apply=False, p=1.4693423255009142e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(107.72723388671875, 107.72723388671875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00012847788269293448, shift_limit_x=(0.3492872714996338, 0.3492872714996338), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.3104694056080973e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.4258551597595215, -0.4258551597595215), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0016757845809544647, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.8216180801391602, 0.8216180801391602), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0002972416724650352, max_holes=16, max_height=275, max_width=275, min_holes=16, min_height=275, min_width=275, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.24158776876976873, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.48151858897498956),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.218134602283221e-11, r_shift_limit=(-207, -207), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.289005213843569e-11, r_shift_limit=(-0.0, 0.0), g_shift_limit=(22, 22), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0006616532045277401, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-243, -243)),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(-0.03846055269241333, -0.03846055269241333), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.482228290775309e-12, brightness_limit=(0, 0), contrast_limit=(3.0456326007843018, 3.0456326007843018), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0008707523098343203, threshold=(61, 61)),
        HorizontalFlip(always_apply=False, p=4.90401912426313e-05),
        VerticalFlip(always_apply=False, p=3.2642780839475805e-11),
        ShiftScaleRotate(always_apply=False, p=0.00015772331466204492, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-125.22209167480469, -125.22209167480469), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.7701107242779663, shift_limit_x=(-0.05140167474746704, -0.05140167474746704), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.530595554676524e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.6823225021362305, -0.6823225021362305), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.629216234777728e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.751215934753418, 8.751215934753418), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.03110329331588e-08, max_holes=16, max_height=523, max_width=523, min_holes=16, min_height=523, min_width=523, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.2115890638596415e-05, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.2281079402293008),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.01706578600236952, r_shift_limit=(-137, -137), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.13260463394729527, r_shift_limit=(-0.0, 0.0), g_shift_limit=(60, 60), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.2517081936759453, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-120, -120)),
        RandomBrightnessContrast(always_apply=False, p=0.0064283424295972225, brightness_limit=(-0.35600948333740234, -0.35600948333740234), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.01365513798620488, brightness_limit=(0, 0), contrast_limit=(5.865621566772461, 5.865621566772461), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00022601894759608984, threshold=(80, 80)),
        HorizontalFlip(always_apply=False, p=1.3744083059403166e-08),
        VerticalFlip(always_apply=False, p=0.0016711411532956133),
        ShiftScaleRotate(always_apply=False, p=1.1801959920956177e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(55.10313415527344, 55.10313415527344), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.234113814367447e-06, shift_limit_x=(0.34981799125671387, 0.34981799125671387), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.0430943501138362e-07, shift_limit_x=(0, 0), shift_limit_y=(-1.0, -1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.819697605175718e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.353870391845703, 7.353870391845703), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0014031121450724937, max_holes=16, max_height=565, max_width=565, min_holes=16, min_height=565, min_width=565, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.3368827439147637, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.23826943869455475),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=2.006273323486344e-08, r_shift_limit=(-144, -144), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.004703299502272884, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-120, -120), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.4840098690969148e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(159, 159)),
        RandomBrightnessContrast(always_apply=False, p=7.001168707780094e-05, brightness_limit=(-0.047820329666137695, -0.047820329666137695), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.003242900129371562, brightness_limit=(0, 0), contrast_limit=(5.80731201171875, 5.80731201171875), brightness_by_max=True),
        Solarize(always_apply=False, p=1.0765151446594343e-06, threshold=(63, 63)),
        HorizontalFlip(always_apply=False, p=4.17111148651926e-06),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=0.0030745800315175875, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-0.0037078857421875, -0.0037078857421875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.0780195947787985e-07, shift_limit_x=(-0.3443695902824402, -0.3443695902824402), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.369451485232515e-08, shift_limit_x=(0, 0), shift_limit_y=(0.261929988861084, 0.261929988861084), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.37247838025226e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.280614852905273, 8.280614852905273), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.236781397284163e-08, max_holes=16, max_height=391, max_width=391, min_holes=16, min_height=391, min_width=391, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.7223184218919556, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.26658305394644477),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.071079607779836e-08, r_shift_limit=(233, 233), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.5891856518420402e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(140, 140), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.16895430159722746, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(63, 63)),
        RandomBrightnessContrast(always_apply=False, p=1.949135252160479e-07, brightness_limit=(0.9507452249526978, 0.9507452249526978), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.852973168814293e-07, brightness_limit=(0, 0), contrast_limit=(6.357415199279785, 6.357415199279785), brightness_by_max=True),
        Solarize(always_apply=False, p=8.822987094857931e-05, threshold=(225, 225)),
        HorizontalFlip(always_apply=False, p=1.601173328657406e-05),
        VerticalFlip(always_apply=False, p=0.000677841181816248),
        ShiftScaleRotate(always_apply=False, p=7.260109575520264e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(31.8369140625, 31.8369140625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0004721747676958432, shift_limit_x=(-0.18591302633285522, -0.18591302633285522), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.991310702999388e-08, shift_limit_x=(0, 0), shift_limit_y=(1.0, 1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.623616001073044e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.625151634216309, 5.625151634216309), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00017814840433450808, max_holes=16, max_height=373, max_width=373, min_holes=16, min_height=373, min_width=373, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.000918435667350323, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8286211513046589),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00038644134828929276, r_shift_limit=(70, 70), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.50354480784428e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-244, -244), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.4541813384500954e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-156, -156)),
        RandomBrightnessContrast(always_apply=False, p=2.252912693538481e-06, brightness_limit=(0.219901442527771, 0.219901442527771), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=4.553603912079329e-06, brightness_limit=(0, 0), contrast_limit=(2.8704497814178467, 2.8704497814178467), brightness_by_max=True),
        Solarize(always_apply=False, p=3.524964507420462e-08, threshold=(195, 195)),
        HorizontalFlip(always_apply=False, p=6.144643685185368e-07),
        VerticalFlip(always_apply=False, p=6.556562009974074e-09),
        ShiftScaleRotate(always_apply=False, p=0.698411638787249, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0.699798583984375, 0.699798583984375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.761396455127439e-07, shift_limit_x=(0.39572131633758545, 0.39572131633758545), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2794415068526097e-06, shift_limit_x=(0, 0), shift_limit_y=(0.5359382629394531, 0.5359382629394531), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.111007623159424e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.002197027206421, 1.002197027206421), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.08292428960859e-11, max_holes=16, max_height=318, max_width=318, min_holes=16, min_height=318, min_width=318, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=5.960526270763092e-10, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.3011845402543617),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.396096554792383e-09, r_shift_limit=(-154, -154), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.9918650543405935e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(21, 21), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.1022146006814405e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(-1.0, -1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0004185606228425398, brightness_limit=(0, 0), contrast_limit=(1.4857518672943115, 1.4857518672943115), brightness_by_max=True),
        Solarize(always_apply=False, p=2.0882625116641366e-08, threshold=(213, 213)),
        HorizontalFlip(always_apply=False, p=2.6351764923738478e-05),
        VerticalFlip(always_apply=False, p=0.0035441459816369647),
        ShiftScaleRotate(always_apply=False, p=5.4838781332335764e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-80.80431365966797, -80.80431365966797), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.003977679048822319, shift_limit_x=(0.34098756313323975, 0.34098756313323975), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.717297967548077e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.5587313175201416, -0.5587313175201416), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2947488104139578e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.265950202941895, 8.265950202941895), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.0778391247962546e-05, max_holes=16, max_height=349, max_width=349, min_holes=16, min_height=349, min_width=349, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.5624363175260605, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.42954621493246037),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=8.590098894132305e-08, r_shift_limit=(149, 149), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9338189716222587e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(150, 150), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.005064911159511132, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-118, -118)),
        RandomBrightnessContrast(always_apply=False, p=0.0020512942095210207, brightness_limit=(0.785150408744812, 0.785150408744812), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.246441805377177e-10, brightness_limit=(0, 0), contrast_limit=(3.2922136783599854, 3.2922136783599854), brightness_by_max=True),
        Solarize(always_apply=False, p=1.9460537432115527e-06, threshold=(234, 234)),
        HorizontalFlip(always_apply=False, p=5.789708480869252e-05),
        VerticalFlip(always_apply=False, p=7.435215278228061e-09),
        ShiftScaleRotate(always_apply=False, p=0.6579269293852263, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(145.0323486328125, 145.0323486328125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.481239226065332e-07, shift_limit_x=(0.46154332160949707, 0.46154332160949707), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5521233818531713e-08, shift_limit_x=(0, 0), shift_limit_y=(0.19180059432983398, 0.19180059432983398), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.397709681416823e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.622280597686768, 7.622280597686768), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.1700451641011225e-08, max_holes=16, max_height=270, max_width=270, min_holes=16, min_height=270, min_width=270, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00312810870421662, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.3317675752873587),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.8334511094546395e-08, r_shift_limit=(65, 65), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.839438532977023e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-175, -175), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.039821485036486415, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(126, 126)),
        RandomBrightnessContrast(always_apply=False, p=1.5869733609015685e-07, brightness_limit=(-0.6191312074661255, -0.6191312074661255), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.603904690459871e-06, brightness_limit=(0, 0), contrast_limit=(3.4693641662597656, 3.4693641662597656), brightness_by_max=True),
        Solarize(always_apply=False, p=7.18056906417134e-05, threshold=(106, 106)),
        HorizontalFlip(always_apply=False, p=0.003250303287455958),
        VerticalFlip(always_apply=False, p=5.657545373948439e-07),
        ShiftScaleRotate(always_apply=False, p=1.367828484783969e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(121.8983154296875, 121.8983154296875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(-0.25276893377304077, -0.25276893377304077), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.101885543860825e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.47520530223846436, -0.47520530223846436), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.231380893088434e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(10.0, 10.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.044827995056097025, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0003319008042743342, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9116105074444796),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.8811042061249455e-07, r_shift_limit=(-157, -157), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.135846823712398e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-56, -56), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.2065992517352075e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-105, -105)),
        RandomBrightnessContrast(always_apply=False, p=1.5616413425697625e-09, brightness_limit=(0.8464978933334351, 0.8464978933334351), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.3426635794547545e-10, brightness_limit=(0, 0), contrast_limit=(2.6628646850585938, 2.6628646850585938), brightness_by_max=True),
        Solarize(always_apply=False, p=2.5703787542852297e-08, threshold=(140, 140)),
        HorizontalFlip(always_apply=False, p=6.282783426310706e-05),
        VerticalFlip(always_apply=False, p=8.378582210309633e-07),
        ShiftScaleRotate(always_apply=False, p=8.358788264702837e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.41191288358141875, shift_limit_x=(0.612633228302002, 0.612633228302002), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.534801096202244e-12, shift_limit_x=(0, 0), shift_limit_y=(0.6353404521942139, 0.6353404521942139), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.2682785987854, 4.2682785987854), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.2450690851925446e-05, max_holes=16, max_height=450, max_width=450, min_holes=16, min_height=450, min_width=450, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.6613953390989795e-07, max_holes=15, max_height=37, max_width=37, min_holes=15, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5879796198998963),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00027002974377731033, r_shift_limit=(166, 166), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.4918691583514924e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-195, -195), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.11384397299351257, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-186, -186)),
        RandomBrightnessContrast(always_apply=False, p=1.5222650442961723e-07, brightness_limit=(0.8003722429275513, 0.8003722429275513), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.026707103755099215, brightness_limit=(0, 0), contrast_limit=(4.959691524505615, 4.959691524505615), brightness_by_max=True),
        Solarize(always_apply=False, p=2.5594602482590333e-07, threshold=(28, 28)),
        HorizontalFlip(always_apply=False, p=1.3224846074582798e-07),
        VerticalFlip(always_apply=False, p=8.371316025441823e-08),
        ShiftScaleRotate(always_apply=False, p=1.5139524585424259e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-120.33985900878906, -120.33985900878906), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2587561209329697e-10, shift_limit_x=(0.6564351320266724, 0.6564351320266724), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0019247162800697e-05, shift_limit_x=(0, 0), shift_limit_y=(0.3639638423919678, 0.3639638423919678), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.07285364992861343, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.191509246826172, 5.191509246826172), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00022219924119382117, max_holes=16, max_height=435, max_width=435, min_holes=16, min_height=435, min_width=435, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.8901214107597795e-06, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7860640778651622),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.05032877574237604, r_shift_limit=(-132, -132), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.062468411039240834, r_shift_limit=(-0.0, 0.0), g_shift_limit=(32, 32), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.864173883514013e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-68, -68)),
        RandomBrightnessContrast(always_apply=False, p=3.7988436048620906e-05, brightness_limit=(0.13338351249694824, 0.13338351249694824), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0008402001331026185, brightness_limit=(0, 0), contrast_limit=(2.2807679176330566, 2.2807679176330566), brightness_by_max=True),
        Solarize(always_apply=False, p=2.9318604584609453e-07, threshold=(95, 95)),
        HorizontalFlip(always_apply=False, p=4.045020550956288e-07),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=5.8726530843507437e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-96.22322082519531, -96.22322082519531), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6464145815994557e-07, shift_limit_x=(-0.495111882686615, -0.495111882686615), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1000393282280323e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.6394436955451965, -0.6394436955451965), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.3515964359540735e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.023719787597656, 6.023719787597656), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.169340351465673e-07, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8862577544357471),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.810102442067427e-05, r_shift_limit=(-208, -208), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.800219112394499e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-255, -255), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.6010476354375224e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(207, 207)),
        RandomBrightnessContrast(always_apply=False, p=2.494449650808722e-09, brightness_limit=(-0.19436931610107422, -0.19436931610107422), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00203673564743774, brightness_limit=(0, 0), contrast_limit=(4.71478271484375, 4.71478271484375), brightness_by_max=True),
        Solarize(always_apply=False, p=4.54100483712252e-07, threshold=(151, 151)),
        HorizontalFlip(always_apply=False, p=2.3958697506061782e-06),
        VerticalFlip(always_apply=False, p=8.780964194336496e-10),
        ShiftScaleRotate(always_apply=False, p=0.1655649162358097, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(39.376953125, 39.376953125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.1302466424118926e-06, shift_limit_x=(-0.39857709407806396, -0.39857709407806396), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.2698774415630396e-08, shift_limit_x=(0, 0), shift_limit_y=(0.6769994497299194, 0.6769994497299194), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.8035754054893704e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.3408639430999756, 3.3408639430999756), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.5052947562159576e-10, max_holes=16, max_height=235, max_width=235, min_holes=16, min_height=235, min_width=235, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.32947870909097e-10, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8323072027062786),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.3849061645909613e-13, r_shift_limit=(12, 12), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-0.0, 0.0), g_shift_limit=(235, 235), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.855934113992316e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(211, 211)),
        RandomBrightnessContrast(always_apply=False, p=1.5978562433166272e-05, brightness_limit=(0.2570779323577881, 0.2570779323577881), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.1703123584944296e-09, brightness_limit=(0, 0), contrast_limit=(7.329161167144775, 7.329161167144775), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00013012689928241367, threshold=(100, 100)),
        HorizontalFlip(always_apply=False, p=4.849238515310059e-10),
        VerticalFlip(always_apply=False, p=3.0127617785762876e-08),
        ShiftScaleRotate(always_apply=False, p=4.795367662782233e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(52.9410400390625, 52.9410400390625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.8505409131679258, shift_limit_x=(-1.0, -1.0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4735870083098874e-11, shift_limit_x=(0, 0), shift_limit_y=(0.0022728443145751953, 0.0022728443145751953), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0092258143536343e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.118220329284668, 8.118220329284668), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.1102587052181423e-08, max_holes=16, max_height=367, max_width=367, min_holes=16, min_height=367, min_width=367, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.0629891544957606e-06, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.14929988758138013),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.2574044657036127e-08, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9790910936132832e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(229, 229), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.016058555287958853, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=1.5843588130337594e-07, brightness_limit=(0.33165550231933594, 0.33165550231933594), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0003708304300644432, brightness_limit=(0, 0), contrast_limit=(6.573843479156494, 6.573843479156494), brightness_by_max=True),
        Solarize(always_apply=False, p=0.002445741642738211, threshold=(155, 155)),
        HorizontalFlip(always_apply=False, p=0.0004534180675815208),
        VerticalFlip(always_apply=False, p=1.2568733513063785e-05),
        ShiftScaleRotate(always_apply=False, p=0.1926926115293881, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.521373849133067e-05, shift_limit_x=(0.006810784339904785, 0.006810784339904785), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1929835338039341e-05, shift_limit_x=(0, 0), shift_limit_y=(0.5868688821792603, 0.5868688821792603), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.145325300963321e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.291029930114746, 4.291029930114746), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00036496092898717636, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=9.430104702643632e-05, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7874590534255459),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=7.08356046266994e-06, r_shift_limit=(-10, -10), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0016676669499480212, r_shift_limit=(-0.0, 0.0), g_shift_limit=(182, 182), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.016655685253590402, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-183, -183)),
        RandomBrightnessContrast(always_apply=False, p=5.150587053932088e-07, brightness_limit=(0.6276669502258301, 0.6276669502258301), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.000846189624823919, brightness_limit=(0, 0), contrast_limit=(3.177264928817749, 3.177264928817749), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00010482089107196821, threshold=(120, 120)),
        HorizontalFlip(always_apply=False, p=0.006602003085990016),
        VerticalFlip(always_apply=False, p=3.0718300995623485e-07),
        ShiftScaleRotate(always_apply=False, p=4.544428182745217e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.730575614532472e-07, shift_limit_x=(-0.4575430750846863, -0.4575430750846863), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.323067652509066e-10, shift_limit_x=(0, 0), shift_limit_y=(0.43965256214141846, 0.43965256214141846), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.240753354617727e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.142137050628662, 2.142137050628662), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.01488010786627747, max_holes=16, max_height=266, max_width=266, min_holes=16, min_height=266, min_width=266, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=7.918794123296257e-07, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9592241694753023),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=7.585711508283967e-10, r_shift_limit=(181, 181), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.8300513326963141e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(98, 98), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.10650990689593964, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-1, -1)),
        RandomBrightnessContrast(always_apply=False, p=0.010422145947813988, brightness_limit=(0.8140754699707031, 0.8140754699707031), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=8.24677670321081e-07, brightness_limit=(0, 0), contrast_limit=(3.4881949424743652, 3.4881949424743652), brightness_by_max=True),
        Solarize(always_apply=False, p=4.395502018155941e-09, threshold=(252, 252)),
        HorizontalFlip(always_apply=False, p=0.008384909580638422),
        VerticalFlip(always_apply=False, p=7.269219327592398e-07),
        ShiftScaleRotate(always_apply=False, p=0.0007844062375796518, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(25.415176391601562, 25.415176391601562), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0033728743394897676, shift_limit_x=(-0.931963324546814, -0.931963324546814), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.414925710468931e-08, shift_limit_x=(0, 0), shift_limit_y=(0.027402520179748535, 0.027402520179748535), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.7861283284024594e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.9792160987854, 4.9792160987854), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.3628958542444133e-05, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.2300438240497158, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6404567309786828),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.896051334849668e-05, r_shift_limit=(-7, -7), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.859964083420782e-11, r_shift_limit=(-0.0, 0.0), g_shift_limit=(160, 160), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0025516798889598513, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(95, 95)),
        RandomBrightnessContrast(always_apply=False, p=1.4678254893673906e-08, brightness_limit=(0.32521605491638184, 0.32521605491638184), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.157918402082072e-08, brightness_limit=(0, 0), contrast_limit=(0.7937389612197876, 0.7937389612197876), brightness_by_max=True),
        Solarize(always_apply=False, p=0.09732284104096323, threshold=(51, 51)),
        HorizontalFlip(always_apply=False, p=0.005684026499757522),
        VerticalFlip(always_apply=False, p=0.004733822161565371),
        ShiftScaleRotate(always_apply=False, p=0.0025826219458087024, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(46.14863586425781, 46.14863586425781), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.4357345744333788e-11, shift_limit_x=(-0.7137458324432373, -0.7137458324432373), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.004087345985080537, shift_limit_x=(0, 0), shift_limit_y=(0.025472640991210938, 0.025472640991210938), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00019412460026692682, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.7940547466278076, 2.7940547466278076), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.716186462459292e-10, max_holes=16, max_height=521, max_width=521, min_holes=16, min_height=521, min_width=521, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.9876733385132866e-05, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8827946640388497),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.61496103956539e-10, r_shift_limit=(110, 110), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.675036241364668e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(109, 109), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9387005023266575e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(204, 204)),
        RandomBrightnessContrast(always_apply=False, p=1.4210869027393301e-05, brightness_limit=(0.4930037260055542, 0.4930037260055542), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0002936815371716861, brightness_limit=(0, 0), contrast_limit=(1.9853451251983643, 1.9853451251983643), brightness_by_max=True),
        Solarize(always_apply=False, p=9.257347974973492e-08, threshold=(84, 84)),
        HorizontalFlip(always_apply=False, p=5.847865839994713e-11),
        VerticalFlip(always_apply=False, p=1.8245465161506163e-10),
        ShiftScaleRotate(always_apply=False, p=3.7989937868139695e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(165.41500854492188, 165.41500854492188), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0823872762323378e-06, shift_limit_x=(-0.8467553853988647, -0.8467553853988647), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(-0.31394869089126587, -0.31394869089126587), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.396184773317349e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.061683654785156, 9.061683654785156), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.60348941654067e-05, max_holes=16, max_height=523, max_width=523, min_holes=16, min_height=523, min_width=523, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.9176208568287265, max_holes=13, max_height=37, max_width=37, min_holes=13, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.08195028626567091),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=2.0179134003209138e-09, r_shift_limit=(-159, -159), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0010852209368364396, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-255, -255), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.001109082894178083, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(58, 58)),
        RandomBrightnessContrast(always_apply=False, p=0.004093328956514597, brightness_limit=(0.9802128076553345, 0.9802128076553345), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.780810793837725e-10, brightness_limit=(0, 0), contrast_limit=(7.551321506500244, 7.551321506500244), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00043973500703718, threshold=(0, 0)),
        HorizontalFlip(always_apply=False, p=0.0),
        VerticalFlip(always_apply=False, p=6.141027655995115e-10),
        ShiftScaleRotate(always_apply=False, p=3.2948193732885037e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(1.252960205078125, 1.252960205078125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.388690297667916e-06, shift_limit_x=(0.7039860486984253, 0.7039860486984253), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.21764069520381213, shift_limit_x=(0, 0), shift_limit_y=(-0.6838634014129639, -0.6838634014129639), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.822629164726612e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.9802727699279785, 4.9802727699279785), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00016215568113233693, max_holes=16, max_height=356, max_width=356, min_holes=16, min_height=356, min_width=356, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0005739049227252851, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7748861485928025),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.1991090763139443e-09, r_shift_limit=(78, 78), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.534070593587025e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(159, 159), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.653578048636157e-11, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(244, 244)),
        RandomBrightnessContrast(always_apply=False, p=1.424726255926656e-07, brightness_limit=(0.7809383869171143, 0.7809383869171143), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.238227953381441e-10, brightness_limit=(0, 0), contrast_limit=(5.402557849884033, 5.402557849884033), brightness_by_max=True),
        Solarize(always_apply=False, p=2.1436227464121868e-05, threshold=(118, 118)),
        HorizontalFlip(always_apply=False, p=0.00015954571622470506),
        VerticalFlip(always_apply=False, p=1.8375441360811434e-09),
        ShiftScaleRotate(always_apply=False, p=0.0008709338585281556, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-124.10528564453125, -124.10528564453125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.8979792813693166, shift_limit_x=(0.26960909366607666, 0.26960909366607666), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.086223092765295e-09, shift_limit_x=(0, 0), shift_limit_y=(0.659049391746521, 0.659049391746521), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.8981438307634175e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.226798057556152, 7.226798057556152), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.3321186385888197e-07, max_holes=16, max_height=375, max_width=375, min_holes=16, min_height=375, min_width=375, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.318563041561292e-08, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.10096814291296752),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.9653713756215787e-05, r_shift_limit=(196, 196), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.988685011487753e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(205, 205), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.105865923426008e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-138, -138)),
        RandomBrightnessContrast(always_apply=False, p=3.795116651389394e-11, brightness_limit=(-0.6260539293289185, -0.6260539293289185), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00013504507305561568, brightness_limit=(0, 0), contrast_limit=(5.414150238037109, 5.414150238037109), brightness_by_max=True),
        Solarize(always_apply=False, p=1.3946552169818694e-11, threshold=(242, 242)),
        HorizontalFlip(always_apply=False, p=0.00016011471277563218),
        VerticalFlip(always_apply=False, p=1.3032701089540338e-07),
        ShiftScaleRotate(always_apply=False, p=1.5285484552666184e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(59.62188720703125, 59.62188720703125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.8547318871352658, shift_limit_x=(-0.6884539127349854, -0.6884539127349854), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.079144324921062e-06, shift_limit_x=(0, 0), shift_limit_y=(0.3761177062988281, 0.3761177062988281), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.602248796179824e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.5202789306640625, 2.5202789306640625), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.8126143048979353e-07, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.427792327857898e-06, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.14491626256280654),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.289324071153389e-06, r_shift_limit=(-40, -40), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.526900792642539e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(105, 105), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.9385935747171477e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(0, 0)),
        RandomBrightnessContrast(always_apply=False, p=3.0623873404792188e-09, brightness_limit=(0.014435052871704102, 0.014435052871704102), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.389564447211975e-05, brightness_limit=(0, 0), contrast_limit=(10.0, 10.0), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0034132361856829274, threshold=(105, 105)),
        HorizontalFlip(always_apply=False, p=2.7370213731452625e-06),
        VerticalFlip(always_apply=False, p=0.0009632439682911442),
        ShiftScaleRotate(always_apply=False, p=0.0011051162611777632, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-5.1935272216796875, -5.1935272216796875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.9687794487130927e-05, shift_limit_x=(0.5953866243362427, 0.5953866243362427), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.749202567235778e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.6628535985946655, -0.6628535985946655), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.032083034515381, 3.032083034515381), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.000778814847529466, max_holes=16, max_height=121, max_width=121, min_holes=16, min_height=121, min_width=121, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.46773854066411324, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5259203785886271),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.012329896252357475, r_shift_limit=(67, 67), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.509929079815028e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(56, 56), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.018037319212306446, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-31, -31)),
        RandomBrightnessContrast(always_apply=False, p=9.624542957595484e-09, brightness_limit=(0.600784182548523, 0.600784182548523), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.007171714050912881, brightness_limit=(0, 0), contrast_limit=(8.948565483093262, 8.948565483093262), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0012004688851171424, threshold=(199, 199)),
        HorizontalFlip(always_apply=False, p=0.005698795988396288),
        VerticalFlip(always_apply=False, p=5.580860564248155e-06),
        ShiftScaleRotate(always_apply=False, p=2.440856621575664e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(106.63299560546875, 106.63299560546875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6450520527997828e-08, shift_limit_x=(-0.8294448256492615, -0.8294448256492615), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5900002705047117e-07, shift_limit_x=(0, 0), shift_limit_y=(-0.8001293540000916, -0.8001293540000916), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.05177944986056948, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.1025707721710205, 2.1025707721710205), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.1786324041779193e-09, max_holes=16, max_height=258, max_width=258, min_holes=16, min_height=258, min_width=258, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.419527631743567e-07, max_holes=13, max_height=37, max_width=37, min_holes=13, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.903656638826276),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.1871349367920602e-08, r_shift_limit=(-155, -155), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.01208141592508749, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-78, -78), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.05536519194717293, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(117, 117)),
        RandomBrightnessContrast(always_apply=False, p=6.491932679425147e-05, brightness_limit=(0.9027066230773926, 0.9027066230773926), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0020431126398053195, brightness_limit=(0, 0), contrast_limit=(3.985750913619995, 3.985750913619995), brightness_by_max=True),
        Solarize(always_apply=False, p=1.7512573521221743e-05, threshold=(36, 36)),
        HorizontalFlip(always_apply=False, p=4.2859004512714113e-08),
        VerticalFlip(always_apply=False, p=1.6523516817510396e-06),
        ShiftScaleRotate(always_apply=False, p=2.464860765474088e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-1.0025177001953125, -1.0025177001953125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.375143529959079e-10, shift_limit_x=(-0.13485628366470337, -0.13485628366470337), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.918161538461233e-07, shift_limit_x=(0, 0), shift_limit_y=(0.16441023349761963, 0.16441023349761963), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.029187479195905e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.677055597305298, 3.677055597305298), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.017675689994716137, max_holes=16, max_height=270, max_width=270, min_holes=16, min_height=270, min_width=270, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.29863173165103873, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6141169210225514),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.1033244164389364e-06, r_shift_limit=(-89, -89), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00030795124913297195, r_shift_limit=(-0.0, 0.0), g_shift_limit=(194, 194), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9013527766207573e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-180, -180)),
        RandomBrightnessContrast(always_apply=False, p=0.006088688010368604, brightness_limit=(-0.721898078918457, -0.721898078918457), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.016369154001970254, brightness_limit=(0, 0), contrast_limit=(3.963243007659912, 3.963243007659912), brightness_by_max=True),
        Solarize(always_apply=False, p=0.1124957128361217, threshold=(246, 246)),
        HorizontalFlip(always_apply=False, p=1.7444904965256278e-09),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=2.3425038786524868e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-27.58734130859375, -27.58734130859375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.262352721944595e-09, shift_limit_x=(0.42738842964172363, 0.42738842964172363), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.822594625641829e-06, shift_limit_x=(0, 0), shift_limit_y=(-0.42341500520706177, -0.42341500520706177), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.880888748292285e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(10.0, 10.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.226833353377255e-07, max_holes=16, max_height=68, max_width=68, min_holes=16, min_height=68, min_width=68, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8647079863599828),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.540583344453643e-09, r_shift_limit=(-211, -211), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.202930286171264e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-195, -195), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.186845022157933e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-97, -97)),
        RandomBrightnessContrast(always_apply=False, p=0.0002434712940038372, brightness_limit=(-0.5205913186073303, -0.5205913186073303), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.377557334067362e-07, brightness_limit=(0, 0), contrast_limit=(3.809776544570923, 3.809776544570923), brightness_by_max=True),
        Solarize(always_apply=False, p=3.0997981303077264e-07, threshold=(135, 135)),
        HorizontalFlip(always_apply=False, p=8.211740860063495e-11),
        VerticalFlip(always_apply=False, p=8.876600458742191e-06),
        ShiftScaleRotate(always_apply=False, p=0.13493587749626101, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(162.69561767578125, 162.69561767578125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.668167479241795e-07, shift_limit_x=(0.5881392955780029, 0.5881392955780029), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00017401321531038214, shift_limit_x=(0, 0), shift_limit_y=(0.4403771162033081, 0.4403771162033081), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.338910482146613e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.738103866577148, 5.738103866577148), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.091330039702159e-09, max_holes=16, max_height=397, max_width=397, min_holes=16, min_height=397, min_width=397, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0017483117888801164, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8628036210672048),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.854772748247455e-08, r_shift_limit=(-84, -84), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.025057475250311878, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-103, -103), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.008591498943121e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(62, 62)),
        RandomBrightnessContrast(always_apply=False, p=5.186278504827832e-09, brightness_limit=(0.9003373384475708, 0.9003373384475708), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.002913758624345064, brightness_limit=(0, 0), contrast_limit=(4.907361030578613, 4.907361030578613), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0, threshold=(100, 100)),
        HorizontalFlip(always_apply=False, p=7.896752396214688e-07),
        VerticalFlip(always_apply=False, p=5.136825209515587e-10),
        ShiftScaleRotate(always_apply=False, p=2.0059699113475683e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(35.18324279785156, 35.18324279785156), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.9171214982252334e-07, shift_limit_x=(0.4037593603134155, 0.4037593603134155), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.3855955052915885, shift_limit_x=(0, 0), shift_limit_y=(-0.44695794582366943, -0.44695794582366943), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.370960630084179e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.681467056274414, 3.681467056274414), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.000274995865765959, max_holes=16, max_height=394, max_width=394, min_holes=16, min_height=394, min_width=394, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.1456267655963494e-06, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5861560861746262),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.025534090275917354, r_shift_limit=(104, 104), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.2322243202062704e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(95, 95), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00031105625057384345, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-206, -206)),
        RandomBrightnessContrast(always_apply=False, p=0.002394226689068149, brightness_limit=(-0.0994347333908081, -0.0994347333908081), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.6636422489272754e-05, brightness_limit=(0, 0), contrast_limit=(2.095242500305176, 2.095242500305176), brightness_by_max=True),
        Solarize(always_apply=False, p=7.539275673888257e-05, threshold=(211, 211)),
        HorizontalFlip(always_apply=False, p=6.868545622419527e-11),
        VerticalFlip(always_apply=False, p=0.3429323897869292),
        ShiftScaleRotate(always_apply=False, p=9.336706453932344e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-53.464019775390625, -53.464019775390625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0017064365902887818, shift_limit_x=(0.3804636001586914, 0.3804636001586914), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.858579979967348e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.1979755163192749, -0.1979755163192749), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.3766685128781984e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.545081615447998, 3.545081615447998), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.67556191812392e-05, max_holes=16, max_height=106, max_width=106, min_holes=16, min_height=106, min_width=106, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.742527481814876e-09, max_holes=14, max_height=37, max_width=37, min_holes=14, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6268596327190212),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.371478118630081e-09, r_shift_limit=(71, 71), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.015702742599928987, r_shift_limit=(-0.0, 0.0), g_shift_limit=(141, 141), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.2440159982830695e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(40, 40)),
        RandomBrightnessContrast(always_apply=False, p=0.07432270156884613, brightness_limit=(-0.025177478790283203, -0.025177478790283203), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.833406593678705e-09, brightness_limit=(0, 0), contrast_limit=(2.4035346508026123, 2.4035346508026123), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0014874145443877773, threshold=(157, 157)),
        HorizontalFlip(always_apply=False, p=2.8188374176234053e-10),
        VerticalFlip(always_apply=False, p=2.593873136180931e-07),
        ShiftScaleRotate(always_apply=False, p=0.0008048301549458958, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-144.7896728515625, -144.7896728515625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.7809118602602428e-08, shift_limit_x=(0.8270707130432129, 0.8270707130432129), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0056430517617856e-07, shift_limit_x=(0, 0), shift_limit_y=(0.2502875328063965, 0.2502875328063965), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.140234161586998e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.474396705627441, 4.474396705627441), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.036017810642349e-07, max_holes=16, max_height=421, max_width=421, min_holes=16, min_height=421, min_width=421, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.24830207733565857, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6593708763111843),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.3147816913028524e-08, r_shift_limit=(-13, -13), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.625762136405869e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-251, -251), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.123804000050069e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=1.752382968917123e-11, brightness_limit=(-0.725956916809082, -0.725956916809082), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0001655223638921139, brightness_limit=(0, 0), contrast_limit=(6.970390796661377, 6.970390796661377), brightness_by_max=True),
        Solarize(always_apply=False, p=2.793327691979285e-08, threshold=(209, 209)),
        HorizontalFlip(always_apply=False, p=2.5652202451246852e-11),
        VerticalFlip(always_apply=False, p=6.339689045815097e-06),
        ShiftScaleRotate(always_apply=False, p=6.18515897041993e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-122.14936828613281, -122.14936828613281), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.691322989971749e-10, shift_limit_x=(0.9302619695663452, 0.9302619695663452), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.8613668366768934, shift_limit_x=(0, 0), shift_limit_y=(-0.7496569156646729, -0.7496569156646729), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.425371370567798e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.9553728103637695, 7.9553728103637695), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.358493122041041e-10, max_holes=16, max_height=211, max_width=211, min_holes=16, min_height=211, min_width=211, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.31012401798998e-07, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.13838861342357434),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.00036563299702170766, r_shift_limit=(-39, -39), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.3817498122111971, r_shift_limit=(-0.0, 0.0), g_shift_limit=(26, 26), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.006472611929844829, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-92, -92)),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(-0.9312046766281128, -0.9312046766281128), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=8.687241694807848e-05, brightness_limit=(0, 0), contrast_limit=(9.171175003051758, 9.171175003051758), brightness_by_max=True),
        Solarize(always_apply=False, p=2.1670878175084003e-07, threshold=(222, 222)),
        HorizontalFlip(always_apply=False, p=0.0004690977200970392),
        VerticalFlip(always_apply=False, p=3.7477668253263645e-07),
        ShiftScaleRotate(always_apply=False, p=1.0663546238500127e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-125.85243225097656, -125.85243225097656), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8941556528490955e-06, shift_limit_x=(-0.0016443729400634766, -0.0016443729400634766), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0058028605646337494, shift_limit_x=(0, 0), shift_limit_y=(-0.00033485889434814453, -0.00033485889434814453), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1449465674223322e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.8770354986190796, 1.8770354986190796), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0003421303306378118, max_holes=16, max_height=101, max_width=101, min_holes=16, min_height=101, min_width=101, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=5.49743185417886e-06, max_holes=14, max_height=37, max_width=37, min_holes=14, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6046808857447357),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00037082497389841207, r_shift_limit=(65, 65), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.002100138821908959, r_shift_limit=(-0.0, 0.0), g_shift_limit=(130, 130), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.1505701373728063e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(80, 80)),
        RandomBrightnessContrast(always_apply=False, p=0.00018318417689671285, brightness_limit=(0.1852656602859497, 0.1852656602859497), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=4.648447206560988e-07, brightness_limit=(0, 0), contrast_limit=(6.884672164916992, 6.884672164916992), brightness_by_max=True),
        Solarize(always_apply=False, p=4.170760874845696e-06, threshold=(68, 68)),
        HorizontalFlip(always_apply=False, p=0.001867187926725078),
        VerticalFlip(always_apply=False, p=2.4146638645458338e-11),
        ShiftScaleRotate(always_apply=False, p=7.822668429705537e-12, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(112.56271362304688, 112.56271362304688), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.583264103528607e-06, shift_limit_x=(0.5773500204086304, 0.5773500204086304), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.49910186074405516, shift_limit_x=(0, 0), shift_limit_y=(0.7947788238525391, 0.7947788238525391), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.3141281227157136e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.552445411682129, 3.552445411682129), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.446394570905974e-06, max_holes=16, max_height=426, max_width=426, min_holes=16, min_height=426, min_width=426, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.1972358413518465e-08, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.49636497420365244),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.2918437582212796, r_shift_limit=(-67, -67), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0651045612496195, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-80, -80), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.347364061962948e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(24, 24)),
        RandomBrightnessContrast(always_apply=False, p=4.698117408778861e-07, brightness_limit=(-0.39438050985336304, -0.39438050985336304), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(0, 0), contrast_limit=(0.9291613698005676, 0.9291613698005676), brightness_by_max=True),
        Solarize(always_apply=False, p=1.4318262011566503e-06, threshold=(98, 98)),
        HorizontalFlip(always_apply=False, p=0.00016055089021606381),
        VerticalFlip(always_apply=False, p=1.8511037617923777e-07),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-65.33358764648438, -65.33358764648438), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.5623908535060344e-07, shift_limit_x=(-0.012957453727722168, -0.012957453727722168), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.2543935785179338e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.44582581520080566, -0.44582581520080566), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.8843590446903794e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.343304634094238, 4.343304634094238), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.455514808824834e-09, max_holes=16, max_height=310, max_width=310, min_holes=16, min_height=310, min_width=310, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.029236065385867427, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6135603995564886),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=7.747949830413885e-07, r_shift_limit=(113, 113), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.006590219501928507, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-73, -73), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.753792327769353e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(78, 78)),
        RandomBrightnessContrast(always_apply=False, p=9.389729507677195e-07, brightness_limit=(-0.6730092167854309, -0.6730092167854309), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.012395033576135805, brightness_limit=(0, 0), contrast_limit=(8.501045227050781, 8.501045227050781), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0022334181796664354, threshold=(103, 103)),
        HorizontalFlip(always_apply=False, p=0.0),
        VerticalFlip(always_apply=False, p=7.902666057276891e-08),
        ShiftScaleRotate(always_apply=False, p=3.418031448873799e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(5.80511474609375, 5.80511474609375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0.1582697629928589, 0.1582697629928589), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2048541265519435e-07, shift_limit_x=(0, 0), shift_limit_y=(0.7825555801391602, 0.7825555801391602), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.001902489341236796, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.257448196411133, 8.257448196411133), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.562040035701833e-08, max_holes=16, max_height=355, max_width=355, min_holes=16, min_height=355, min_width=355, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.044147242195784e-06, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9768753670123147),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.06646329539449702, r_shift_limit=(118, 118), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0034587898936173644, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-159, -159), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.3543857204263355e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(27, 27)),
        RandomBrightnessContrast(always_apply=False, p=0.0076302336698782225, brightness_limit=(-0.36735886335372925, -0.36735886335372925), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.6338231117017854e-08, brightness_limit=(0, 0), contrast_limit=(1.919584035873413, 1.919584035873413), brightness_by_max=True),
        Solarize(always_apply=False, p=0.08405914870767406, threshold=(7, 7)),
        HorizontalFlip(always_apply=False, p=0.0034773302707815645),
        VerticalFlip(always_apply=False, p=2.112743674999914e-05),
        ShiftScaleRotate(always_apply=False, p=1.757604394528973e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(52.2216796875, 52.2216796875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.1381455600076555e-08, shift_limit_x=(-0.0989232063293457, -0.0989232063293457), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.895002546685903e-07, shift_limit_x=(0, 0), shift_limit_y=(-0.16864436864852905, -0.16864436864852905), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.03968825041352653, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.660511016845703, 6.660511016845703), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=8.694526607994006e-08, max_holes=16, max_height=16, max_width=16, min_holes=16, min_height=16, min_width=16, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=5.942337650208934e-05, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7951195562419),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.721437768481363e-10, r_shift_limit=(141, 141), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.303279298603036e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-255, -255), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.6384201831563007e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(34, 34)),
        RandomBrightnessContrast(always_apply=False, p=1.2551806002119507e-07, brightness_limit=(0.2303178310394287, 0.2303178310394287), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.988223436453947e-09, brightness_limit=(0, 0), contrast_limit=(0.8331835865974426, 0.8331835865974426), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0, threshold=(169, 169)),
        HorizontalFlip(always_apply=False, p=1.0760787815155667e-07),
        VerticalFlip(always_apply=False, p=6.496118792965523e-07),
        ShiftScaleRotate(always_apply=False, p=0.0013401682121564618, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(63.15458679199219, 63.15458679199219), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.9642538456222667, shift_limit_x=(-0.1429300308227539, -0.1429300308227539), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.642326176695205e-06, shift_limit_x=(0, 0), shift_limit_y=(0.004754781723022461, 0.004754781723022461), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4731231557357088e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.287440299987793, 4.287440299987793), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.8642429111969426e-09, max_holes=16, max_height=459, max_width=459, min_holes=16, min_height=459, min_width=459, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=8.77065892522732e-09, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.034385702806008056),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.984126966146366e-06, r_shift_limit=(162, 162), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.754867005080031e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-119, -119), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.070155957826986e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(60, 60)),
        RandomBrightnessContrast(always_apply=False, p=0.00035180892001942035, brightness_limit=(0.3076355457305908, 0.3076355457305908), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00013543546508943544, brightness_limit=(0, 0), contrast_limit=(0.3106270432472229, 0.3106270432472229), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0014264143780530203, threshold=(84, 84)),
        HorizontalFlip(always_apply=False, p=2.3532429499896305e-07),
        VerticalFlip(always_apply=False, p=1.510895915065008e-09),
        ShiftScaleRotate(always_apply=False, p=2.654348298065372e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(71.37738037109375, 71.37738037109375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.73558624777592e-07, shift_limit_x=(0.21770036220550537, 0.21770036220550537), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.614463361587488e-07, shift_limit_x=(0, 0), shift_limit_y=(-4.988908767700195e-05, -4.988908767700195e-05), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.415321334227589e-12, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.742497682571411, 2.742497682571411), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.002020599140637805, max_holes=16, max_height=345, max_width=345, min_holes=16, min_height=345, min_width=345, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.9886794686317444, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.007378557378147743),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00024632923520482436, r_shift_limit=(-171, -171), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.026565922083517712, r_shift_limit=(-0.0, 0.0), g_shift_limit=(92, 92), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.007262908044221028, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-198, -198)),
        RandomBrightnessContrast(always_apply=False, p=0.008623472537155508, brightness_limit=(-0.5726301670074463, -0.5726301670074463), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0006748028594591622, brightness_limit=(0, 0), contrast_limit=(10.0, 10.0), brightness_by_max=True),
        Solarize(always_apply=False, p=0.006903780210011146, threshold=(94, 94)),
        HorizontalFlip(always_apply=False, p=1.7828259364630663e-06),
        VerticalFlip(always_apply=False, p=5.454808591329063e-09),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-111.42981719970703, -111.42981719970703), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0005950652908636583, shift_limit_x=(-0.42745524644851685, -0.42745524644851685), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.303380250129131e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.3022769093513489, -0.3022769093513489), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.01818238679281059, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.611422061920166, 2.611422061920166), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.42492599612076987, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.418334054934428e-05, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5059833221708896),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.10956210214581308, r_shift_limit=(10, 10), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.130355963867329e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-130, -130), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.12952387785936992, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(99, 99)),
        RandomBrightnessContrast(always_apply=False, p=0.053022250130893234, brightness_limit=(0.07210016250610352, 0.07210016250610352), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.315672368633436e-05, brightness_limit=(0, 0), contrast_limit=(7.066825866699219, 7.066825866699219), brightness_by_max=True),
        Solarize(always_apply=False, p=6.118318945047291e-05, threshold=(157, 157)),
        HorizontalFlip(always_apply=False, p=8.616928490105195e-06),
        VerticalFlip(always_apply=False, p=1.5555108453083221e-09),
        ShiftScaleRotate(always_apply=False, p=1.5108669073943173e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-64.8057632446289, -64.8057632446289), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.007504237006551451, shift_limit_x=(-0.701224684715271, -0.701224684715271), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.326771824344407e-06, shift_limit_x=(0, 0), shift_limit_y=(-1.0, -1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.9250107215329415e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.909184455871582, 4.909184455871582), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00042973542843087464, max_holes=16, max_height=26, max_width=26, min_holes=16, min_height=26, min_width=26, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00035422620997578244, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6994507545238936),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=8.336354614972127e-12, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.354347358649695e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-219, -219), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.13336272637999e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-219, -219)),
        RandomBrightnessContrast(always_apply=False, p=2.4418202612492693e-08, brightness_limit=(0.7896828651428223, 0.7896828651428223), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0005708706462353892, brightness_limit=(0, 0), contrast_limit=(3.951472759246826, 3.951472759246826), brightness_by_max=True),
        Solarize(always_apply=False, p=2.7815574518082892e-05, threshold=(196, 196)),
        HorizontalFlip(always_apply=False, p=1.2683733878084656e-07),
        VerticalFlip(always_apply=False, p=0.0004594643290506059),
        ShiftScaleRotate(always_apply=False, p=0.11975036108751569, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(1.8523406982421875, 1.8523406982421875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0003423461952053661, shift_limit_x=(-0.7451590895652771, -0.7451590895652771), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.831861433880303e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.442018985748291, -0.442018985748291), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.008518305917332e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.349853515625, 9.349853515625), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.148149977235886e-05, max_holes=16, max_height=550, max_width=550, min_holes=16, min_height=550, min_width=550, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.0306817143404773e-05, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8787764003477346),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.4158024610261045e-10, r_shift_limit=(-5, -5), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.4400462390428216e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-139, -139), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9546653796258326e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(200, 200)),
        RandomBrightnessContrast(always_apply=False, p=1.6703586000352483e-06, brightness_limit=(-0.6164284348487854, -0.6164284348487854), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=7.270698991679848e-07, brightness_limit=(0, 0), contrast_limit=(9.119046211242676, 9.119046211242676), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00018903678549139682, threshold=(169, 169)),
        HorizontalFlip(always_apply=False, p=0.08377210096015109),
        VerticalFlip(always_apply=False, p=1.9419446421371913e-06),
        ShiftScaleRotate(always_apply=False, p=1.7432006027442268e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(108.83956909179688, 108.83956909179688), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.985409650352447e-09, shift_limit_x=(0.000738978385925293, 0.000738978385925293), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0013002757887137645, shift_limit_x=(0, 0), shift_limit_y=(0.6059273481369019, 0.6059273481369019), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00023046225120997108, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.242457866668701, 7.242457866668701), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0, max_holes=16, max_height=203, max_width=203, min_holes=16, min_height=203, min_width=203, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.4627890455184196e-09, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9145003279666073),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0160840926034993, r_shift_limit=(87, 87), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.201556933819581e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(84, 84), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.06568376529695996, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(167, 167)),
        RandomBrightnessContrast(always_apply=False, p=2.0231202240667666e-09, brightness_limit=(0.14635074138641357, 0.14635074138641357), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.040108586377368e-08, brightness_limit=(0, 0), contrast_limit=(1.240447998046875, 1.240447998046875), brightness_by_max=True),
        Solarize(always_apply=False, p=0.033596337444899094, threshold=(172, 172)),
        HorizontalFlip(always_apply=False, p=3.972048709487272e-06),
        VerticalFlip(always_apply=False, p=6.977550093267837e-09),
        ShiftScaleRotate(always_apply=False, p=3.029819140600281e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(48.783477783203125, 48.783477783203125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.5678599738671844e-05, shift_limit_x=(0.7125002145767212, 0.7125002145767212), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0020614802235921215, shift_limit_x=(0, 0), shift_limit_y=(-1.0, -1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.427191513488728e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.114917278289795, 1.114917278289795), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.1876011936789925e-09, max_holes=16, max_height=260, max_width=260, min_holes=16, min_height=260, min_width=260, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.3018590246573168e-07, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8825341625826666),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.0038823287356537506, r_shift_limit=(113, 113), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.02501558074086141, r_shift_limit=(-0.0, 0.0), g_shift_limit=(37, 37), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0012931111843516863, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(102, 102)),
        RandomBrightnessContrast(always_apply=False, p=0.24546443488121916, brightness_limit=(0.010127663612365723, 0.010127663612365723), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.258412523224547e-06, brightness_limit=(0, 0), contrast_limit=(9.113080024719238, 9.113080024719238), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0, threshold=(0, 0)),
        HorizontalFlip(always_apply=False, p=5.8046495729361985e-06),
        VerticalFlip(always_apply=False, p=2.032377299120281e-09),
        ShiftScaleRotate(always_apply=False, p=0.018217879347275145, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-165.66868591308594, -165.66868591308594), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00039653875944064415, shift_limit_x=(-0.7849082350730896, -0.7849082350730896), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.264270388399329e-05, shift_limit_x=(0, 0), shift_limit_y=(1.0, 1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.0845155733030984e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.577235221862793, 9.577235221862793), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.988100761018535e-08, max_holes=16, max_height=463, max_width=463, min_holes=16, min_height=463, min_width=463, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.03821494068041664, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6674511195398591),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.55294123032702e-10, r_shift_limit=(-28, -28), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.524650128076584e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-8, -8), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0021441313095440367, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(8, 8)),
        RandomBrightnessContrast(always_apply=False, p=7.860922836318019e-08, brightness_limit=(-0.14497894048690796, -0.14497894048690796), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.138198868065018e-06, brightness_limit=(0, 0), contrast_limit=(2.168576240539551, 2.168576240539551), brightness_by_max=True),
        Solarize(always_apply=False, p=4.8634178808515394e-05, threshold=(3, 3)),
        HorizontalFlip(always_apply=False, p=1.4803544382085395e-11),
        VerticalFlip(always_apply=False, p=6.3509170577175245e-12),
        ShiftScaleRotate(always_apply=False, p=8.805346862061767e-12, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-2.3247833251953125, -2.3247833251953125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.7799323472857367e-10, shift_limit_x=(0.5677356719970703, 0.5677356719970703), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.462099148004161e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.23122787475585938, -0.23122787475585938), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.757517753558777e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.7853941917419434, 1.7853941917419434), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.0242945742683956e-06, max_holes=16, max_height=103, max_width=103, min_holes=16, min_height=103, min_width=103, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.5867807701934922, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4110124639358089),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.003138125471719072, r_shift_limit=(16, 16), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.5823040637259088e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-92, -92), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.6602839127210183e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(223, 223)),
        RandomBrightnessContrast(always_apply=False, p=4.7466833958849595e-09, brightness_limit=(0.7070457935333252, 0.7070457935333252), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.014201578684151173, brightness_limit=(0, 0), contrast_limit=(7.556657791137695, 7.556657791137695), brightness_by_max=True),
        Solarize(always_apply=False, p=1.4668322865236481e-07, threshold=(139, 139)),
        HorizontalFlip(always_apply=False, p=0.0022870561466524775),
        VerticalFlip(always_apply=False, p=0.0014202680975894805),
        ShiftScaleRotate(always_apply=False, p=1.8699574088906688e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-19.97955322265625, -19.97955322265625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.682210153840763e-11, shift_limit_x=(-0.8060465455055237, -0.8060465455055237), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.21782766574006018, shift_limit_x=(0, 0), shift_limit_y=(0.5211914777755737, 0.5211914777755737), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.6415292501132203e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.310616493225098, 4.310616493225098), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00017420021782882378, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.7200921854465604e-07, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7609231965173857),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.022662974449521478, r_shift_limit=(6, 6), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.3399307857034115e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(18, 18), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.7276038978805198e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-23, -23)),
        RandomBrightnessContrast(always_apply=False, p=6.389725306222942e-07, brightness_limit=(0.9227428436279297, 0.9227428436279297), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=8.172217098356737e-07, brightness_limit=(0, 0), contrast_limit=(5.941784858703613, 5.941784858703613), brightness_by_max=True),
        Solarize(always_apply=False, p=1.1921161476992992e-08, threshold=(133, 133)),
        HorizontalFlip(always_apply=False, p=8.745805006654935e-07),
        VerticalFlip(always_apply=False, p=1.4237919198801196e-09),
        ShiftScaleRotate(always_apply=False, p=5.0427229834817e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-86.04209899902344, -86.04209899902344), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.662461590873431e-08, shift_limit_x=(-0.8074343204498291, -0.8074343204498291), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.510914688902743e-10, shift_limit_x=(0, 0), shift_limit_y=(1.0, 1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0204288243405055e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.9882200956344604, 0.9882200956344604), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.742915237114056e-07, max_holes=16, max_height=493, max_width=493, min_holes=16, min_height=493, min_width=493, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.8740020008094547, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.10333122803859685),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=3.876322612217201e-10, r_shift_limit=(222, 222), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.3212753921746506e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-206, -206), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00083677716330538, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(146, 146)),
        RandomBrightnessContrast(always_apply=False, p=2.99327970042847e-08, brightness_limit=(0.5300668478012085, 0.5300668478012085), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.2281125591271835e-10, brightness_limit=(0, 0), contrast_limit=(1.9315705299377441, 1.9315705299377441), brightness_by_max=True),
        Solarize(always_apply=False, p=4.927910777026065e-09, threshold=(244, 244)),
        HorizontalFlip(always_apply=False, p=1.287649943779497e-07),
        VerticalFlip(always_apply=False, p=1.4250708251424068e-10),
        ShiftScaleRotate(always_apply=False, p=0.43350483276751106, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-74.30584716796875, -74.30584716796875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.7352822827975573e-10, shift_limit_x=(0.6562174558639526, 0.6562174558639526), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.322400070215845e-08, shift_limit_x=(0, 0), shift_limit_y=(0.0663372278213501, 0.0663372278213501), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8655857758102952e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.302128791809082, 6.302128791809082), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.5483396281086668e-05, max_holes=16, max_height=572, max_width=572, min_holes=16, min_height=572, min_width=572, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.3683821088601723e-07, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.56561067334683),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0022143353601574017, r_shift_limit=(-77, -77), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.04474291593041002, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-135, -135), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.3741892371773403, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-106, -106)),
        RandomBrightnessContrast(always_apply=False, p=0.00016428211302680837, brightness_limit=(-1.0, -1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00801419143227311, brightness_limit=(0, 0), contrast_limit=(1.5784066915512085, 1.5784066915512085), brightness_by_max=True),
        Solarize(always_apply=False, p=1.9672536544949583e-09, threshold=(21, 21)),
        HorizontalFlip(always_apply=False, p=0.00040635850160817763),
        VerticalFlip(always_apply=False, p=0.015186839097549587),
        ShiftScaleRotate(always_apply=False, p=0.0005372689844718379, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-3.8000640869140625, -3.8000640869140625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.004647598563022909, shift_limit_x=(1.0, 1.0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.000407020725904296, shift_limit_x=(0, 0), shift_limit_y=(-0.6689871549606323, -0.6689871549606323), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.1299193581933613, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.433409690856934, 5.433409690856934), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.654618461197694e-09, max_holes=16, max_height=198, max_width=198, min_holes=16, min_height=198, min_width=198, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.5554507433759785e-06, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4195680268482588),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0031096869172469455, r_shift_limit=(216, 216), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.121658947711529e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(63, 63), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.3441336800255712, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-175, -175)),
        RandomBrightnessContrast(always_apply=False, p=3.993945653576591e-06, brightness_limit=(0.389743447303772, 0.389743447303772), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.004007036620129745, brightness_limit=(0, 0), contrast_limit=(2.9445078372955322, 2.9445078372955322), brightness_by_max=True),
        Solarize(always_apply=False, p=0.052283430928188324, threshold=(91, 91)),
        HorizontalFlip(always_apply=False, p=0.013157062622924931),
        VerticalFlip(always_apply=False, p=1.2367724058464226e-08),
        ShiftScaleRotate(always_apply=False, p=0.0014381061670214468, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-30.706329345703125, -30.706329345703125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.10864164162287793, shift_limit_x=(-0.6393939256668091, -0.6393939256668091), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00010978329316202809, shift_limit_x=(0, 0), shift_limit_y=(0.003858327865600586, 0.003858327865600586), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.934811454366186e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.466071128845215, 4.466071128845215), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00017957173669636006, max_holes=16, max_height=233, max_width=233, min_holes=16, min_height=233, min_width=233, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.8808906403508334e-08, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4728947576609387),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00914640103547043, r_shift_limit=(76, 76), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.95720385110641e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-121, -121), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.7516867842508223e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-92, -92)),
        RandomBrightnessContrast(always_apply=False, p=0.09559316642011018, brightness_limit=(1.0, 1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0029601068511286854, brightness_limit=(0, 0), contrast_limit=(7.434629440307617, 7.434629440307617), brightness_by_max=True),
        Solarize(always_apply=False, p=0.007269729276558057, threshold=(52, 52)),
        HorizontalFlip(always_apply=False, p=0.0003126311214876609),
        VerticalFlip(always_apply=False, p=3.08148100941711e-05),
        ShiftScaleRotate(always_apply=False, p=8.370169005702243e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-69.67778015136719, -69.67778015136719), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.625198386645423e-10, shift_limit_x=(1.0, 1.0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.849286025289759e-10, shift_limit_x=(0, 0), shift_limit_y=(0.8770685195922852, 0.8770685195922852), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2240439482211e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.725141525268555, 5.725141525268555), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=8.938190622006144e-08, max_holes=16, max_height=307, max_width=307, min_holes=16, min_height=307, min_width=307, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.7597410651518999, max_holes=16, max_height=37, max_width=37, min_holes=16, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.12493776162362114),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.00020776784446216098, r_shift_limit=(82, 82), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.021089695911477024, r_shift_limit=(-0.0, 0.0), g_shift_limit=(17, 17), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.2273976237896215e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(197, 197)),
        RandomBrightnessContrast(always_apply=False, p=0.00010096573894437938, brightness_limit=(0.028228163719177246, 0.028228163719177246), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.452177126997638e-06, brightness_limit=(0, 0), contrast_limit=(1.3591421842575073, 1.3591421842575073), brightness_by_max=True),
        Solarize(always_apply=False, p=1.9442003021922252e-06, threshold=(192, 192)),
        HorizontalFlip(always_apply=False, p=1.4815935847253532e-06),
        VerticalFlip(always_apply=False, p=0.0006713862940050802),
        ShiftScaleRotate(always_apply=False, p=0.07636029613848638, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(70.15873718261719, 70.15873718261719), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2665594631967284e-08, shift_limit_x=(0.546432375907898, 0.546432375907898), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00016060024582965904, shift_limit_x=(0, 0), shift_limit_y=(-0.29632681608200073, -0.29632681608200073), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6192395787313962e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.034128189086914, 8.034128189086914), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.695173248920611e-06, max_holes=16, max_height=319, max_width=319, min_holes=16, min_height=319, min_width=319, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=7.195338861525416e-05, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9013205257266362),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00010949711571916609, r_shift_limit=(10, 10), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0004309355910531165, r_shift_limit=(-0.0, 0.0), g_shift_limit=(145, 145), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.0090051409447723e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-170, -170)),
        RandomBrightnessContrast(always_apply=False, p=0.012342694604536852, brightness_limit=(-0.415455162525177, -0.415455162525177), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.0650412704070206e-05, brightness_limit=(0, 0), contrast_limit=(10.0, 10.0), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00016780221256529337, threshold=(105, 105)),
        HorizontalFlip(always_apply=False, p=0.2900888234556831),
        VerticalFlip(always_apply=False, p=2.5150089914664515e-06),
        ShiftScaleRotate(always_apply=False, p=2.1666870972793744e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(64.97331237792969, 64.97331237792969), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.866946821697638e-07, shift_limit_x=(-0.4629848599433899, -0.4629848599433899), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0009361345226925472, shift_limit_x=(0, 0), shift_limit_y=(0.8171955347061157, 0.8171955347061157), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2547566834237764e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.4940836429595947, 3.4940836429595947), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00022950167779802952, max_holes=16, max_height=450, max_width=450, min_holes=16, min_height=450, min_width=450, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.498806871984542e-05, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6956509554804088),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.430997256527148e-07, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.849297779502486e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-19, -19), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.49209965986895554, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(0, 0)),
        RandomBrightnessContrast(always_apply=False, p=1.1017382858481958e-06, brightness_limit=(-0.459585964679718, -0.459585964679718), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0038464068582625965, brightness_limit=(0, 0), contrast_limit=(1.3277082443237305, 1.3277082443237305), brightness_by_max=True),
        Solarize(always_apply=False, p=0.008213081414408407, threshold=(81, 81)),
        HorizontalFlip(always_apply=False, p=1.7494006212229829e-09),
        VerticalFlip(always_apply=False, p=0.00016675456203820924),
        ShiftScaleRotate(always_apply=False, p=8.560644476026602e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-146.1626434326172, -146.1626434326172), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4531967560230185e-08, shift_limit_x=(-0.008427143096923828, -0.008427143096923828), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.002605235230639047, shift_limit_x=(0, 0), shift_limit_y=(-0.347898006439209, -0.347898006439209), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1467394745587675e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.8048062324523926, 1.8048062324523926), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.435025647857396e-09, max_holes=16, max_height=212, max_width=212, min_holes=16, min_height=212, min_width=212, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=9.360046369938444e-08, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4930658520719301),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.012333619107805838, r_shift_limit=(14, 14), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.4156711548403643e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-34, -34), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.44277991083659707, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(35, 35)),
        RandomBrightnessContrast(always_apply=False, p=5.348392878544066e-08, brightness_limit=(-0.09859418869018555, -0.09859418869018555), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.029425196333665582, brightness_limit=(0, 0), contrast_limit=(1.1625771522521973, 1.1625771522521973), brightness_by_max=True),
        Solarize(always_apply=False, p=2.9949249643466104e-10, threshold=(143, 143)),
        HorizontalFlip(always_apply=False, p=2.0190168139456813e-10),
        VerticalFlip(always_apply=False, p=0.0007540396629612985),
        ShiftScaleRotate(always_apply=False, p=0.00014707707501399395, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(61.2608642578125, 61.2608642578125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.427985218318557e-07, shift_limit_x=(0.25046610832214355, 0.25046610832214355), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0005246113889675122, shift_limit_x=(0, 0), shift_limit_y=(-0.0037796497344970703, -0.0037796497344970703), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.2891519999145883e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.0073916912078857, 1.0073916912078857), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.711046040289896e-05, max_holes=16, max_height=219, max_width=219, min_holes=16, min_height=219, min_width=219, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0007795209836599648, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5131652116903704),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=3.602645036381659e-06, r_shift_limit=(208, 208), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00031784209445410647, r_shift_limit=(-0.0, 0.0), g_shift_limit=(39, 39), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.1076472506347135e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-42, -42)),
        RandomBrightnessContrast(always_apply=False, p=0.00651958996355817, brightness_limit=(0.22846674919128418, 0.22846674919128418), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=4.8678969037645615e-05, brightness_limit=(0, 0), contrast_limit=(5.964810848236084, 5.964810848236084), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0002335656064170119, threshold=(35, 35)),
        HorizontalFlip(always_apply=False, p=0.0001701388168643585),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=1.2942051464876653e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(143.61776733398438, 143.61776733398438), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0912491079366424e-08, shift_limit_x=(-0.24281960725784302, -0.24281960725784302), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0.20073676109313965, 0.20073676109313965), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8260948092107743e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.9281044006347656, 3.9281044006347656), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.005676578750991063, max_holes=16, max_height=237, max_width=237, min_holes=16, min_height=237, min_width=237, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.44287745050562677, max_holes=16, max_height=37, max_width=37, min_holes=16, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.544152519072723),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0020651961269196606, r_shift_limit=(200, 200), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.6252662340400384e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-166, -166), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.204501744753129, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-45, -45)),
        RandomBrightnessContrast(always_apply=False, p=5.91168061185466e-05, brightness_limit=(-0.5336875319480896, -0.5336875319480896), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=7.362172648232423e-06, brightness_limit=(0, 0), contrast_limit=(1.318678379058838, 1.318678379058838), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00012679816900644514, threshold=(163, 163)),
        HorizontalFlip(always_apply=False, p=1.7866645176627856e-07),
        VerticalFlip(always_apply=False, p=0.005345849721909213),
        ShiftScaleRotate(always_apply=False, p=2.3462292870602525e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-163.98501586914062, -163.98501586914062), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.3094339600732968e-09, shift_limit_x=(-0.8048264384269714, -0.8048264384269714), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8709854814917124e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.4370921850204468, -0.4370921850204468), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.8552499283963924e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.694210052490234, 7.694210052490234), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.2190591195197238e-05, max_holes=16, max_height=150, max_width=150, min_holes=16, min_height=150, min_width=150, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00025445270556605573, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7876147460230489),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.9075637692350824, r_shift_limit=(-39, -39), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.426837001871061e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(121, 121), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.018379882441395434, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-101, -101)),
        RandomBrightnessContrast(always_apply=False, p=2.161943823059592e-06, brightness_limit=(0.2001286745071411, 0.2001286745071411), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0028947640445528, brightness_limit=(0, 0), contrast_limit=(6.838469982147217, 6.838469982147217), brightness_by_max=True),
        Solarize(always_apply=False, p=7.910885864618827e-06, threshold=(28, 28)),
        HorizontalFlip(always_apply=False, p=3.463187276225093e-08),
        VerticalFlip(always_apply=False, p=1.568183272127044e-08),
        ShiftScaleRotate(always_apply=False, p=7.252502839417285e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-97.53553009033203, -97.53553009033203), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0001603093576384683, shift_limit_x=(1.0, 1.0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4051299612525658e-07, shift_limit_x=(0, 0), shift_limit_y=(0.4351717233657837, 0.4351717233657837), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.251881940177442e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.065166473388672, 5.065166473388672), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0006394053139896078, max_holes=16, max_height=199, max_width=199, min_holes=16, min_height=199, min_width=199, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.024154338796677166, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.04615602165203192),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.19730222374185757, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.006154058597497514, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-143, -143), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.003090166877743883, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-57, -57)),
        RandomBrightnessContrast(always_apply=False, p=7.072286618593353e-09, brightness_limit=(0.40188419818878174, 0.40188419818878174), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.5164472764023884, brightness_limit=(0, 0), contrast_limit=(0.9209418892860413, 0.9209418892860413), brightness_by_max=True),
        Solarize(always_apply=False, p=2.3782875424860881e-07, threshold=(137, 137)),
        HorizontalFlip(always_apply=False, p=3.397450553937754e-06),
        VerticalFlip(always_apply=False, p=7.095363211920568e-09),
        ShiftScaleRotate(always_apply=False, p=3.6670100903420317e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(53.73295593261719, 53.73295593261719), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.03483158187871371, shift_limit_x=(0.43345439434051514, 0.43345439434051514), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2944734092227544e-10, shift_limit_x=(0, 0), shift_limit_y=(0.5085853338241577, 0.5085853338241577), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.3597000420870994e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.99999993922529e-09, 9.99999993922529e-09), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.5357752323701297e-05, max_holes=16, max_height=432, max_width=432, min_holes=16, min_height=432, min_width=432, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.0003056952862535e-07, max_holes=16, max_height=37, max_width=37, min_holes=16, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.24215558083952016),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.000253407352393549, r_shift_limit=(-139, -139), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0011727709667578232, r_shift_limit=(-0.0, 0.0), g_shift_limit=(29, 29), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00010598716755564567, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=3.6462244969235995e-07, brightness_limit=(0.9741086959838867, 0.9741086959838867), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00040676503629546307, brightness_limit=(0, 0), contrast_limit=(3.864189624786377, 3.864189624786377), brightness_by_max=True),
        Solarize(always_apply=False, p=7.727090393372382e-05, threshold=(58, 58)),
        HorizontalFlip(always_apply=False, p=9.998723330503505e-09),
        VerticalFlip(always_apply=False, p=1.6233505784018609e-09),
        ShiftScaleRotate(always_apply=False, p=2.0372653228882065e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-83.27046966552734, -83.27046966552734), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00020450341965032662, shift_limit_x=(0.40812504291534424, 0.40812504291534424), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.018392487416277525, shift_limit_x=(0, 0), shift_limit_y=(0.024628758430480957, 0.024628758430480957), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.6148268640966634, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(10.0, 10.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.056023912528422315, max_holes=16, max_height=336, max_width=336, min_holes=16, min_height=336, min_width=336, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0013381567952548343, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.3071954608069488),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.0471966852257445e-09, r_shift_limit=(63, 63), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.280215858896777e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-50, -50), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.429312598532088e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(137, 137)),
        RandomBrightnessContrast(always_apply=False, p=1.00464692454492e-08, brightness_limit=(-0.029388606548309326, -0.029388606548309326), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0008813456320809765, brightness_limit=(0, 0), contrast_limit=(3.8411130905151367, 3.8411130905151367), brightness_by_max=True),
        Solarize(always_apply=False, p=7.2943475424516524e-09, threshold=(44, 44)),
        HorizontalFlip(always_apply=False, p=0.00032995886447511474),
        VerticalFlip(always_apply=False, p=6.211078234606957e-10),
        ShiftScaleRotate(always_apply=False, p=0.0005468825889715367, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(157.39215087890625, 157.39215087890625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.5310656469595175e-07, shift_limit_x=(-0.014447391033172607, -0.014447391033172607), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.548841988140147, shift_limit_x=(0, 0), shift_limit_y=(0.23694205284118652, 0.23694205284118652), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.286866148973825e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.99999993922529e-09, 9.99999993922529e-09), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00021821127804802498, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.2230521381140962e-05, max_holes=13, max_height=37, max_width=37, min_holes=13, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4491648092719138),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=7.981613128446455e-05, r_shift_limit=(164, 164), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.6071193000996925e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-112, -112), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.7370180508575754e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=6.016125429931685e-09, brightness_limit=(1.0, 1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.8716383143892333e-08, brightness_limit=(0, 0), contrast_limit=(0.8102353811264038, 0.8102353811264038), brightness_by_max=True),
        Solarize(always_apply=False, p=3.2801285688608436e-09, threshold=(45, 45)),
        HorizontalFlip(always_apply=False, p=1.3048889094552932e-09),
        VerticalFlip(always_apply=False, p=3.1214264034578355e-09),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(129.0181884765625, 129.0181884765625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.4110823970663233, shift_limit_x=(0.6220138072967529, 0.6220138072967529), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0006756236596131837, shift_limit_x=(0, 0), shift_limit_y=(-0.3838122487068176, -0.3838122487068176), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0012066585673742858, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.143016815185547, 5.143016815185547), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.8226792464475237e-05, max_holes=16, max_height=520, max_width=520, min_holes=16, min_height=520, min_width=520, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.5046906681146364e-11, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.586927233421211),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=6.853290846446885e-06, r_shift_limit=(224, 224), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.612780325844309e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(22, 22), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.3250609376239434e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-69, -69)),
        RandomBrightnessContrast(always_apply=False, p=2.0710156627395146e-05, brightness_limit=(-1.0, -1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.737496157185364e-07, brightness_limit=(0, 0), contrast_limit=(0.5341580510139465, 0.5341580510139465), brightness_by_max=True),
        Solarize(always_apply=False, p=1.1197135948288408e-08, threshold=(8, 8)),
        HorizontalFlip(always_apply=False, p=3.114069767037528e-05),
        VerticalFlip(always_apply=False, p=1.7168783098740788e-11),
        ShiftScaleRotate(always_apply=False, p=1.5367169443267215e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-73.1436767578125, -73.1436767578125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.9620931262845083e-07, shift_limit_x=(-0.766445517539978, -0.766445517539978), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(-0.49947673082351685, -0.49947673082351685), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.541988819640026e-12, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.867910861968994, 4.867910861968994), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.0632943477306105e-10, max_holes=16, max_height=179, max_width=179, min_holes=16, min_height=179, min_width=179, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.7215426027832521, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.27836919229289936),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.022798949755006293, r_shift_limit=(139, 139), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.419677597231365e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-92, -92), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.004650141139804054, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(43, 43)),
        RandomBrightnessContrast(always_apply=False, p=5.071775422937062e-08, brightness_limit=(-0.5348035097122192, -0.5348035097122192), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=8.540191327036702e-06, brightness_limit=(0, 0), contrast_limit=(8.950645446777344, 8.950645446777344), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0016551392980979768, threshold=(245, 245)),
        HorizontalFlip(always_apply=False, p=4.981654459551277e-11),
        VerticalFlip(always_apply=False, p=1.0661918345424067e-07),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-143.082275390625, -143.082275390625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.22610419798338e-09, shift_limit_x=(0.29237985610961914, 0.29237985610961914), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00010265455817533645, shift_limit_x=(0, 0), shift_limit_y=(-0.827390730381012, -0.827390730381012), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2794360247924426e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.0241650342941284, 1.0241650342941284), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.002045516257224378, max_holes=16, max_height=448, max_width=448, min_holes=16, min_height=448, min_width=448, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.4526458146205167e-08, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9686318875248281),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=9.39028314952562e-05, r_shift_limit=(226, 226), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-98, -98), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.266241796161782e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(107, 107)),
        RandomBrightnessContrast(always_apply=False, p=4.458517867170469e-07, brightness_limit=(-0.6806083917617798, -0.6806083917617798), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.0911647824012456e-06, brightness_limit=(0, 0), contrast_limit=(5.083920478820801, 5.083920478820801), brightness_by_max=True),
        Solarize(always_apply=False, p=2.8577574897743595e-06, threshold=(184, 184)),
        HorizontalFlip(always_apply=False, p=1.0060480514552841e-09),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=0.00010153706273555597, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-180.0, -180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.1728755239058295e-07, shift_limit_x=(0.6686937808990479, 0.6686937808990479), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.3951891334038643e-07, shift_limit_x=(0, 0), shift_limit_y=(0.7452609539031982, 0.7452609539031982), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.0076947227805525e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.575815200805664, 2.575815200805664), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.4203592010847225e-09, max_holes=16, max_height=448, max_width=448, min_holes=16, min_height=448, min_width=448, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.8349301902924537, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.16479665238072727),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.5839092967883466e-06, r_shift_limit=(-113, -113), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.888520168647566e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-29, -29), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9190041932934377e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-110, -110)),
        RandomBrightnessContrast(always_apply=False, p=6.197743333199476e-08, brightness_limit=(-1.0, -1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=7.391809063042187e-09, brightness_limit=(0, 0), contrast_limit=(2.7145376205444336, 2.7145376205444336), brightness_by_max=True),
        Solarize(always_apply=False, p=0.5929337703359892, threshold=(249, 249)),
        HorizontalFlip(always_apply=False, p=1.3709374785984835e-08),
        VerticalFlip(always_apply=False, p=1.1647486598814615e-08),
        ShiftScaleRotate(always_apply=False, p=1.471410517929577e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(73.11506652832031, 73.11506652832031), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.915917483647419e-11, shift_limit_x=(-0.0012435317039489746, -0.0012435317039489746), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8090476304411955e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.7331608533859253, -0.7331608533859253), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.987382761840015e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.034008026123047, 6.034008026123047), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.6536164234410187e-06, max_holes=16, max_height=256, max_width=256, min_holes=16, min_height=256, min_width=256, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.764313511562058e-09, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4070540850903823),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=6.139915868102504e-12, r_shift_limit=(226, 226), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.893323316717236e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(225, 225), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.004581890652367165, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-24, -24)),
        RandomBrightnessContrast(always_apply=False, p=7.71011835699613e-09, brightness_limit=(-0.27809327840805054, -0.27809327840805054), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.006294470964987586, brightness_limit=(0, 0), contrast_limit=(4.7359771728515625, 4.7359771728515625), brightness_by_max=True),
        Solarize(always_apply=False, p=1.0773499940759443e-08, threshold=(76, 76)),
        HorizontalFlip(always_apply=False, p=7.702350047880061e-06),
        VerticalFlip(always_apply=False, p=0.2881354245465104),
        ShiftScaleRotate(always_apply=False, p=1.0160900994031496e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.998816948329928e-08, shift_limit_x=(-0.2659444212913513, -0.2659444212913513), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.656249979758842e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.26243215799331665, -0.26243215799331665), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.556104588221614e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.448959350585938, 9.448959350585938), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.6462315872777253e-05, max_holes=16, max_height=229, max_width=229, min_holes=16, min_height=229, min_width=229, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.03248618282767679, max_holes=12, max_height=37, max_width=37, min_holes=12, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6684575322086529),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.2206550690128465e-08, r_shift_limit=(-125, -125), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.0156006958003214e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(1, 1), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.235439764350496e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-173, -173)),
        RandomBrightnessContrast(always_apply=False, p=4.910716847384057e-07, brightness_limit=(0.14615535736083984, 0.14615535736083984), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.958966009436519e-10, brightness_limit=(0, 0), contrast_limit=(0.3533366620540619, 0.3533366620540619), brightness_by_max=True),
        Solarize(always_apply=False, p=5.867279956549469e-07, threshold=(124, 124)),
        HorizontalFlip(always_apply=False, p=2.4308777366510706e-07),
        VerticalFlip(always_apply=False, p=0.0008228369035361005),
        ShiftScaleRotate(always_apply=False, p=8.332173645233234e-13, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(93.15362548828125, 93.15362548828125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.3441447609925987, shift_limit_x=(0.10889947414398193, 0.10889947414398193), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.293436098054551e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.15082472562789917, -0.15082472562789917), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.109010365389152e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.098080635070801, 6.098080635070801), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0, max_holes=16, max_height=55, max_width=55, min_holes=16, min_height=55, min_width=55, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.2271674349376664e-06, max_holes=2, max_height=37, max_width=37, min_holes=2, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6550288032565776),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.852256164414784e-09, r_shift_limit=(216, 216), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.901886901787034e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(176, 176), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.135318477828029e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-255, -255)),
        RandomBrightnessContrast(always_apply=False, p=6.084944564509404e-06, brightness_limit=(0.6825240850448608, 0.6825240850448608), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00011778379410984016, brightness_limit=(0, 0), contrast_limit=(1.5145373344421387, 1.5145373344421387), brightness_by_max=True),
        Solarize(always_apply=False, p=1.7330654124997784e-10, threshold=(147, 147)),
        HorizontalFlip(always_apply=False, p=0.29521991610036835),
        VerticalFlip(always_apply=False, p=3.5761411181548484e-06),
        ShiftScaleRotate(always_apply=False, p=2.7623877750605546e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(122.36981201171875, 122.36981201171875), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.190486151080805e-09, shift_limit_x=(0.5087019205093384, 0.5087019205093384), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.783359657597839e-07, shift_limit_x=(0, 0), shift_limit_y=(-0.464078426361084, -0.464078426361084), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8645843456849796e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.617980718612671, 3.617980718612671), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=8.099666834896003e-09, max_holes=16, max_height=181, max_width=181, min_holes=16, min_height=181, min_width=181, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.2416678390989857e-09, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7045383132177867),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.2985788102402915e-06, r_shift_limit=(-136, -136), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.3216968492743035e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(100, 100), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.642833579955673e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(30, 30)),
        RandomBrightnessContrast(always_apply=False, p=0.010371003692171699, brightness_limit=(-0.724097490310669, -0.724097490310669), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.015247940193232279, brightness_limit=(0, 0), contrast_limit=(6.425212860107422, 6.425212860107422), brightness_by_max=True),
        Solarize(always_apply=False, p=3.962012631508749e-05, threshold=(100, 100)),
        HorizontalFlip(always_apply=False, p=0.0056050819495129245),
        VerticalFlip(always_apply=False, p=0.0),
        ShiftScaleRotate(always_apply=False, p=6.810986405025068e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(66.61518859863281, 66.61518859863281), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2840428284571452e-07, shift_limit_x=(0.7357066869735718, 0.7357066869735718), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.7181750822360354e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.26709556579589844, -0.26709556579589844), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.976814637526823e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.130609035491943, 6.130609035491943), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.142842094995467e-08, max_holes=16, max_height=33, max_width=33, min_holes=16, min_height=33, min_width=33, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.18325673532408615, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7854755121089616),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.9370922318420123e-06, r_shift_limit=(183, 183), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.96716137416754e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-216, -216), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.46181962470792e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-31, -31)),
        RandomBrightnessContrast(always_apply=False, p=7.323407149579755e-07, brightness_limit=(-0.5140019655227661, -0.5140019655227661), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.363495175089e-08, brightness_limit=(0, 0), contrast_limit=(1.1973129510879517, 1.1973129510879517), brightness_by_max=True),
        Solarize(always_apply=False, p=9.152837056003562e-09, threshold=(130, 130)),
        HorizontalFlip(always_apply=False, p=1.0990531773471396e-05),
        VerticalFlip(always_apply=False, p=0.00046497867271677595),
        ShiftScaleRotate(always_apply=False, p=0.4112415855086127, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-150.70985412597656, -150.70985412597656), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.006274952614987728, shift_limit_x=(0.8746788501739502, 0.8746788501739502), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4519316768254206e-06, shift_limit_x=(0, 0), shift_limit_y=(0.5338551998138428, 0.5338551998138428), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.5446888222657085e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.801212787628174, 4.801212787628174), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.004516207103849729, max_holes=16, max_height=451, max_width=451, min_holes=16, min_height=451, min_width=451, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.3838046958641002e-09, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5774321773568338),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.0005504579704604107, r_shift_limit=(-99, -99), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.817178985005264e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(255, 255), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.942667253112702e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-217, -217)),
        RandomBrightnessContrast(always_apply=False, p=1.7026100373851757e-09, brightness_limit=(0.6009364128112793, 0.6009364128112793), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.080645707886561e-09, brightness_limit=(0, 0), contrast_limit=(1.6989541053771973, 1.6989541053771973), brightness_by_max=True),
        Solarize(always_apply=False, p=0.9349170209894169, threshold=(103, 103)),
        HorizontalFlip(always_apply=False, p=2.515328762644803e-05),
        VerticalFlip(always_apply=False, p=1.2457636513079562e-09),
        ShiftScaleRotate(always_apply=False, p=1.1548286280818287e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(130.40420532226562, 130.40420532226562), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.101378167335639e-11, shift_limit_x=(0.1425788402557373, 0.1425788402557373), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(-0.22447842359542847, -0.22447842359542847), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.6754304588100464e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.9969388246536255, 0.9969388246536255), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.228015144289255e-07, max_holes=16, max_height=200, max_width=200, min_holes=16, min_height=200, min_width=200, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.38888425226733e-05, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.06441421838725658),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=8.543728049775953e-08, r_shift_limit=(-1, -1), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00020407179604424106, r_shift_limit=(-0.0, 0.0), g_shift_limit=(23, 23), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.07952743907172266, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(133, 133)),
        RandomBrightnessContrast(always_apply=False, p=7.133846340322356e-06, brightness_limit=(0.6548956632614136, 0.6548956632614136), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.4965812183327678, brightness_limit=(0, 0), contrast_limit=(2.3857738971710205, 2.3857738971710205), brightness_by_max=True),
        Solarize(always_apply=False, p=0.001842098644494694, threshold=(155, 155)),
        HorizontalFlip(always_apply=False, p=0.0),
        VerticalFlip(always_apply=False, p=2.2482975625929362e-07),
        ShiftScaleRotate(always_apply=False, p=1.1861389621887037e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0.786407470703125, 0.786407470703125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.4882463372632307e-06, shift_limit_x=(0.8478924036026001, 0.8478924036026001), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.744557710446755e-09, shift_limit_x=(0, 0), shift_limit_y=(0.028117060661315918, 0.028117060661315918), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.270102245524637e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(7.870881080627441, 7.870881080627441), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00012812368915877254, max_holes=16, max_height=475, max_width=475, min_holes=16, min_height=475, min_width=475, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0011915109459195439, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.42051495479149925),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.3064045676696585e-08, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.114564935528918, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-33, -33), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.011844099222521054, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(39, 39)),
        RandomBrightnessContrast(always_apply=False, p=0.0014247934790015404, brightness_limit=(-0.6497026681900024, -0.6497026681900024), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.108356894981579e-07, brightness_limit=(0, 0), contrast_limit=(2.509491205215454, 2.509491205215454), brightness_by_max=True),
        Solarize(always_apply=False, p=4.720202710116819e-05, threshold=(237, 237)),
        HorizontalFlip(always_apply=False, p=3.231779095428972e-05),
        VerticalFlip(always_apply=False, p=2.177489729554805e-08),
        ShiftScaleRotate(always_apply=False, p=0.0051194655867889405, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-158.65756225585938, -158.65756225585938), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.580524460000224e-10, shift_limit_x=(0.3510688543319702, 0.3510688543319702), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.712479171191587e-06, shift_limit_x=(0, 0), shift_limit_y=(-0.767142653465271, -0.767142653465271), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.682081545110248e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.291869640350342, 5.291869640350342), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.682755058538988e-07, max_holes=16, max_height=175, max_width=175, min_holes=16, min_height=175, min_width=175, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.41070212856676136, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.45626166438977633),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=6.214653846108595e-07, r_shift_limit=(142, 142), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00044877641239496335, r_shift_limit=(-0.0, 0.0), g_shift_limit=(23, 23), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.005223684927312233, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-194, -194)),
        RandomBrightnessContrast(always_apply=False, p=3.675670215237442e-05, brightness_limit=(-0.04946267604827881, -0.04946267604827881), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.5460940286666567e-07, brightness_limit=(0, 0), contrast_limit=(3.0114219188690186, 3.0114219188690186), brightness_by_max=True),
        Solarize(always_apply=False, p=0.28938093451892755, threshold=(177, 177)),
        HorizontalFlip(always_apply=False, p=0.022313778268791884),
        VerticalFlip(always_apply=False, p=3.838343192884308e-06),
        ShiftScaleRotate(always_apply=False, p=2.531999824460825e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.1533161291274574e-08, shift_limit_x=(-0.8018327355384827, -0.8018327355384827), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0007971139560492002, shift_limit_x=(0, 0), shift_limit_y=(0.681587815284729, 0.681587815284729), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0004321961102823034, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.110210418701172, 8.110210418701172), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.2861404135347714e-08, max_holes=16, max_height=495, max_width=495, min_holes=16, min_height=495, min_width=495, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.1690591618494075, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5123029059101364),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.162373723832019e-05, r_shift_limit=(161, 161), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.0574976226063973e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(169, 169), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.779698714092544e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(68, 68)),
        RandomBrightnessContrast(always_apply=False, p=0.00024318290205499124, brightness_limit=(-0.19524693489074707, -0.19524693489074707), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.415044356736294e-10, brightness_limit=(0, 0), contrast_limit=(6.769630432128906, 6.769630432128906), brightness_by_max=True),
        Solarize(always_apply=False, p=9.62567596588963e-06, threshold=(141, 141)),
        HorizontalFlip(always_apply=False, p=3.031455896901114e-10),
        VerticalFlip(always_apply=False, p=7.637371075056389e-08),
        ShiftScaleRotate(always_apply=False, p=0.7797621373612458, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-180.0, -180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0003719860461420116, shift_limit_x=(-0.5991668701171875, -0.5991668701171875), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1599225600978034e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.821853756904602, -0.821853756904602), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002304312089752153, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.38733172416687, 2.38733172416687), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0025413536347689003, max_holes=16, max_height=462, max_width=462, min_holes=16, min_height=462, min_width=462, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00018220475186458973, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.2166463094261396),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.009087029637362187, r_shift_limit=(55, 55), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.013759931750802257, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-56, -56), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.7002437417156244e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(4, 4)),
        RandomBrightnessContrast(always_apply=False, p=6.500718847351171e-05, brightness_limit=(-0.3363528847694397, -0.3363528847694397), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.1310637547227885e-07, brightness_limit=(0, 0), contrast_limit=(6.797417640686035, 6.797417640686035), brightness_by_max=True),
        Solarize(always_apply=False, p=0.41777680536324624, threshold=(251, 251)),
        HorizontalFlip(always_apply=False, p=0.00012069715419163116),
        VerticalFlip(always_apply=False, p=1.10079385377903e-09),
        ShiftScaleRotate(always_apply=False, p=1.5415004325550186e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(172.1680908203125, 172.1680908203125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4163950512333276e-08, shift_limit_x=(-0.5162754654884338, -0.5162754654884338), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.6949346247162702e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.0057811737060546875, -0.0057811737060546875), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.034070343309015e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.8278374671936035, 5.8278374671936035), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.17362524932221923, max_holes=16, max_height=19, max_width=19, min_holes=16, min_height=19, min_width=19, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.994239901667027e-07, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.38553726721771797),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.002641322581733649, r_shift_limit=(152, 152), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.52334061898475e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-207, -207), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.686261723403892e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(146, 146)),
        RandomBrightnessContrast(always_apply=False, p=1.5012958727871246e-06, brightness_limit=(-0.4465141296386719, -0.4465141296386719), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.440419386786384e-09, brightness_limit=(0, 0), contrast_limit=(0.7972739934921265, 0.7972739934921265), brightness_by_max=True),
        Solarize(always_apply=False, p=8.436403601528458e-10, threshold=(108, 108)),
        HorizontalFlip(always_apply=False, p=7.429313905686323e-11),
        VerticalFlip(always_apply=False, p=2.7476344476189997e-05),
        ShiftScaleRotate(always_apply=False, p=8.987951462907607e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(8.616607666015625, 8.616607666015625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00016947078951183234, shift_limit_x=(0.35666728019714355, 0.35666728019714355), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5662213785559405e-06, shift_limit_x=(0, 0), shift_limit_y=(-0.7387099266052246, -0.7387099266052246), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00015022747768775584, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.458248138427734, 6.458248138427734), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.005397513974457979, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.4837717231247325, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5078133434107649),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=8.079270804390294e-10, r_shift_limit=(195, 195), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.455516703788786e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(158, 158), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-23, -23)),
        RandomBrightnessContrast(always_apply=False, p=5.196204907202489e-06, brightness_limit=(0.22977817058563232, 0.22977817058563232), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.1002933559872948e-07, brightness_limit=(0, 0), contrast_limit=(8.972331047058105, 8.972331047058105), brightness_by_max=True),
        Solarize(always_apply=False, p=1.771404402836773e-06, threshold=(208, 208)),
        HorizontalFlip(always_apply=False, p=0.0),
        VerticalFlip(always_apply=False, p=2.364910825392618e-10),
        ShiftScaleRotate(always_apply=False, p=1.0223857744966864e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(109.16989135742188, 109.16989135742188), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.898640692634456e-05, shift_limit_x=(-0.41875964403152466, -0.41875964403152466), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.9976188540458679, shift_limit_x=(0, 0), shift_limit_y=(-0.3256521224975586, -0.3256521224975586), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.566662160445664e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.504209518432617, 5.504209518432617), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.4223793377509058e-08, max_holes=16, max_height=237, max_width=237, min_holes=16, min_height=237, min_width=237, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.095861175499883e-05, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.0022595518235033474),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=1.1846360683823013e-05, r_shift_limit=(137, 137), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.006158726737271025, r_shift_limit=(-0.0, 0.0), g_shift_limit=(3, 3), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.5644577840623164e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(130, 130)),
        RandomBrightnessContrast(always_apply=False, p=0.008374408621670604, brightness_limit=(0.1167440414428711, 0.1167440414428711), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.9674104447699345e-07, brightness_limit=(0, 0), contrast_limit=(2.845197916030884, 2.845197916030884), brightness_by_max=True),
        Solarize(always_apply=False, p=2.502941336383263e-07, threshold=(131, 131)),
        HorizontalFlip(always_apply=False, p=6.738484732378479e-07),
        VerticalFlip(always_apply=False, p=1.1867015678136456e-05),
        ShiftScaleRotate(always_apply=False, p=2.4496080127425316e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-109.59932708740234, -109.59932708740234), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.2221781477287843, shift_limit_x=(-0.2935255169868469, -0.2935255169868469), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.54662118796246e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.2755810618400574, -0.2755810618400574), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.7843355821054765e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.7591159343719482, 3.7591159343719482), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=8.772493554675294e-08, max_holes=16, max_height=441, max_width=441, min_holes=16, min_height=441, min_width=441, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=4.787135841609357e-09, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7632637287364358),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.5129818524700829e-05, r_shift_limit=(248, 248), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.002672753120757043, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-126, -126), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.420479491313251e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-192, -192)),
        RandomBrightnessContrast(always_apply=False, p=0.0001691706548394515, brightness_limit=(-1.0, -1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.599558537763892e-06, brightness_limit=(0, 0), contrast_limit=(1.960985541343689, 1.960985541343689), brightness_by_max=True),
        Solarize(always_apply=False, p=9.910753129237421e-09, threshold=(190, 190)),
        HorizontalFlip(always_apply=False, p=1.3488750042056048e-08),
        VerticalFlip(always_apply=False, p=2.875082388224961e-06),
        ShiftScaleRotate(always_apply=False, p=1.2191724507562933e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-95.79322052001953, -95.79322052001953), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.135864333535777e-09, shift_limit_x=(0.15941870212554932, 0.15941870212554932), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.6132751018270266e-10, shift_limit_x=(0, 0), shift_limit_y=(-1.0, -1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.514587889243149e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.922539710998535, 4.922539710998535), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.88657015513773e-05, max_holes=16, max_height=250, max_width=250, min_holes=16, min_height=250, min_width=250, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.7585030988126462, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.23855847086294946),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.3295589641674415e-09, r_shift_limit=(-230, -230), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.5629076654191162e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(109, 109), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.186192173495988e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(6, 6)),
        RandomBrightnessContrast(always_apply=False, p=2.632681003578167e-10, brightness_limit=(0.6608633995056152, 0.6608633995056152), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.540483391663452e-09, brightness_limit=(0, 0), contrast_limit=(9.125590324401855, 9.125590324401855), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00024508682428584123, threshold=(82, 82)),
        HorizontalFlip(always_apply=False, p=0.8212954948854865),
        VerticalFlip(always_apply=False, p=2.430275413299153e-10),
        ShiftScaleRotate(always_apply=False, p=6.803744194774899e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-117.0367431640625, -117.0367431640625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2023886920548536e-06, shift_limit_x=(-0.21645575761795044, -0.21645575761795044), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002004997709437878, shift_limit_x=(0, 0), shift_limit_y=(1.0, 1.0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.521028995513916, 2.521028995513916), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.211727509860304e-10, max_holes=16, max_height=546, max_width=546, min_holes=16, min_height=546, min_width=546, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.3077635507444402e-10, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.17825722671194644),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.152058297861559e-06, r_shift_limit=(-22, -22), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0006534452152030107, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-163, -163), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00010727490516306351, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(49, 49)),
        RandomBrightnessContrast(always_apply=False, p=1.958732219326801e-07, brightness_limit=(-0.28664445877075195, -0.28664445877075195), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.05789219080549124, brightness_limit=(0, 0), contrast_limit=(3.64601993560791, 3.64601993560791), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00639849063351583, threshold=(133, 133)),
        HorizontalFlip(always_apply=False, p=1.4431500479471966e-09),
        VerticalFlip(always_apply=False, p=4.2436284409536746e-07),
        ShiftScaleRotate(always_apply=False, p=5.191057586663179e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.002089401173038727, shift_limit_x=(-0.6943753957748413, -0.6943753957748413), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.7762174646163602e-05, shift_limit_x=(0, 0), shift_limit_y=(0.859689474105835, 0.859689474105835), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0028100453208015352, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.560728073120117, 6.560728073120117), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.6181260191270696e-06, max_holes=16, max_height=93, max_width=93, min_holes=16, min_height=93, min_width=93, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=8.214434430260374e-09, max_holes=13, max_height=37, max_width=37, min_holes=13, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9300127986365863),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=8.090860934797949e-08, r_shift_limit=(191, 191), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.115493289562695e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(172, 172), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.6747637095898244e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(17, 17)),
        RandomBrightnessContrast(always_apply=False, p=2.9205494779783146e-06, brightness_limit=(0.4688100814819336, 0.4688100814819336), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00024386016996305777, brightness_limit=(0, 0), contrast_limit=(6.895814895629883, 6.895814895629883), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0007637207168223123, threshold=(133, 133)),
        HorizontalFlip(always_apply=False, p=3.647644630832723e-06),
        VerticalFlip(always_apply=False, p=1.6010173211307253e-10),
        ShiftScaleRotate(always_apply=False, p=3.1779650187058476e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-61.96351623535156, -61.96351623535156), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0.1308962106704712, 0.1308962106704712), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.2550994030167608, shift_limit_x=(0, 0), shift_limit_y=(0.21363139152526855, 0.21363139152526855), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0003496612074767435, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.227897644042969, 8.227897644042969), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=6.6339116959851085e-09, max_holes=16, max_height=123, max_width=123, min_holes=16, min_height=123, min_width=123, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.007915176622425912, max_holes=3, max_height=37, max_width=37, min_holes=3, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7356214226875994),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-132, -132), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00014103808107673518, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-16, -16), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.839443296281997e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=2.718712384814151e-05, brightness_limit=(-0.7050094604492188, -0.7050094604492188), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.0686871770079134e-08, brightness_limit=(0, 0), contrast_limit=(7.990593910217285, 7.990593910217285), brightness_by_max=True),
        Solarize(always_apply=False, p=7.568841172592524e-07, threshold=(235, 235)),
        HorizontalFlip(always_apply=False, p=2.1758030213726897e-06),
        VerticalFlip(always_apply=False, p=6.4011081642294265e-06),
        ShiftScaleRotate(always_apply=False, p=1.6992080472771727e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-0.88800048828125, -0.88800048828125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.68987719475515e-09, shift_limit_x=(-0.7202759981155396, -0.7202759981155396), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2593778853016453e-10, shift_limit_x=(0, 0), shift_limit_y=(0.6490775346755981, 0.6490775346755981), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002005490909963107, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.955005645751953, 2.955005645751953), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.300296093619454e-07, max_holes=16, max_height=489, max_width=489, min_holes=16, min_height=489, min_width=489, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.23337886512561568, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7662342278154874),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(105, 105), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=8.999799617755401e-12, r_shift_limit=(-0.0, 0.0), g_shift_limit=(55, 55), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.752429160782186e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(89, 89)),
        RandomBrightnessContrast(always_apply=False, p=0.0015169511742229372, brightness_limit=(0.508475661277771, 0.508475661277771), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00017012129959914635, brightness_limit=(0, 0), contrast_limit=(1.5737197399139404, 1.5737197399139404), brightness_by_max=True),
        Solarize(always_apply=False, p=9.831048228408817e-08, threshold=(155, 155)),
        HorizontalFlip(always_apply=False, p=2.777391742789434e-10),
        VerticalFlip(always_apply=False, p=4.816819694754802e-05),
        ShiftScaleRotate(always_apply=False, p=0.5905147096358974, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(4.8977203369140625, 4.8977203369140625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.9427319526812895e-08, shift_limit_x=(-0.9473657608032227, -0.9473657608032227), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.700711677005456e-11, shift_limit_x=(0, 0), shift_limit_y=(-0.3155394196510315, -0.3155394196510315), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.860809906767581e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.852278232574463, 4.852278232574463), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=7.211459806944223e-10, max_holes=16, max_height=353, max_width=353, min_holes=16, min_height=353, min_width=353, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.783583992006114e-07, max_holes=15, max_height=37, max_width=37, min_holes=15, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.40765348016156444),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.012400846446183111, r_shift_limit=(-42, -42), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.000523162447717e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-246, -246), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.010858251071699243, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(70, 70)),
        RandomBrightnessContrast(always_apply=False, p=0.0013908582855021168, brightness_limit=(-0.19397985935211182, -0.19397985935211182), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.06204615145483672, brightness_limit=(0, 0), contrast_limit=(2.1890976428985596, 2.1890976428985596), brightness_by_max=True),
        Solarize(always_apply=False, p=0.14235146328900328, threshold=(169, 169)),
        HorizontalFlip(always_apply=False, p=0.004698651519955632),
        VerticalFlip(always_apply=False, p=0.00014020746758608427),
        ShiftScaleRotate(always_apply=False, p=5.116733077463855e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-64.26161193847656, -64.26161193847656), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.1918682719018814e-08, shift_limit_x=(0.7895630598068237, 0.7895630598068237), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.3274016760864149e-05, shift_limit_x=(0, 0), shift_limit_y=(0.2591317892074585, 0.2591317892074585), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00020487373671761872, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.0844526290893555, 6.0844526290893555), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.10696113565437e-10, max_holes=16, max_height=524, max_width=524, min_holes=16, min_height=524, min_width=524, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.005078166926332894, max_holes=16, max_height=37, max_width=37, min_holes=16, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.760812098022443),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.00014466710629559149, r_shift_limit=(-54, -54), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.1000561052974406e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-105, -105), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.4266201518543038e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-230, -230)),
        RandomBrightnessContrast(always_apply=False, p=4.1041527358407935e-07, brightness_limit=(-0.5761220455169678, -0.5761220455169678), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.0062827380722103e-10, brightness_limit=(0, 0), contrast_limit=(9.6019926071167, 9.6019926071167), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00031538433981714853, threshold=(214, 214)),
        HorizontalFlip(always_apply=False, p=1.1546991258149417e-07),
        VerticalFlip(always_apply=False, p=0.44510091116040407),
        ShiftScaleRotate(always_apply=False, p=1.0370010525398475e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(172.73641967773438, 172.73641967773438), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.989336572705845e-09, shift_limit_x=(0.0036742687225341797, 0.0036742687225341797), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.99144304741793e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.7487937211990356, -0.7487937211990356), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.413445667361972e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.327626705169678, 4.327626705169678), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.0, max_holes=16, max_height=504, max_width=504, min_holes=16, min_height=504, min_width=504, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=8.70393092151857e-08, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5544324633971598),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.09028119295022163, r_shift_limit=(-72, -72), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.231466663363839e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-104, -104), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00010248395360275796, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-62, -62)),
        RandomBrightnessContrast(always_apply=False, p=4.0879613294064973e-07, brightness_limit=(-0.4661882519721985, -0.4661882519721985), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.343745163844584e-06, brightness_limit=(0, 0), contrast_limit=(6.993351459503174, 6.993351459503174), brightness_by_max=True),
        Solarize(always_apply=False, p=0.002883581250316125, threshold=(204, 204)),
        HorizontalFlip(always_apply=False, p=7.873667434845331e-06),
        VerticalFlip(always_apply=False, p=3.354668518682373e-08),
        ShiftScaleRotate(always_apply=False, p=3.541095742105025e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(134.99142456054688, 134.99142456054688), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4107698956696794e-07, shift_limit_x=(0.3772655725479126, 0.3772655725479126), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.006064031745163567, shift_limit_x=(0, 0), shift_limit_y=(-0.18989837169647217, -0.18989837169647217), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.547281744502293e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.019951820373535, 8.019951820373535), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=9.697631511727408e-09, max_holes=16, max_height=325, max_width=325, min_holes=16, min_height=325, min_width=325, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=1.1886969170917907e-09, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.9006507776898078),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.857026167521259e-07, r_shift_limit=(26, 26), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.005832565769044118, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-199, -199), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.906466635774644e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-80, -80)),
        RandomBrightnessContrast(always_apply=False, p=0.0, brightness_limit=(-0.3899446725845337, -0.3899446725845337), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.002112971498301794, brightness_limit=(0, 0), contrast_limit=(6.56829309463501, 6.56829309463501), brightness_by_max=True),
        Solarize(always_apply=False, p=2.2139779696865773e-07, threshold=(186, 186)),
        HorizontalFlip(always_apply=False, p=5.744156114915039e-09),
        VerticalFlip(always_apply=False, p=1.1599601825620475e-06),
        ShiftScaleRotate(always_apply=False, p=9.245247641071775e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-122.71830749511719, -122.71830749511719), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.001959466060297274, shift_limit_x=(0.619850754737854, 0.619850754737854), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.663199828449827e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.9133252501487732, -0.9133252501487732), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.6608725067001728, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.4638804197311401, 1.4638804197311401), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.0657226911148145e-07, max_holes=16, max_height=547, max_width=547, min_holes=16, min_height=547, min_width=547, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.002703860554339327, max_holes=14, max_height=37, max_width=37, min_holes=14, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.3264195349433986),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0002907392574164197, r_shift_limit=(216, 216), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.353830687674456e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(178, 178), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.710224865611364e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-236, -236)),
        RandomBrightnessContrast(always_apply=False, p=4.0308280439072756e-05, brightness_limit=(0.34994471073150635, 0.34994471073150635), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.2189293639769483e-07, brightness_limit=(0, 0), contrast_limit=(7.983994960784912, 7.983994960784912), brightness_by_max=True),
        Solarize(always_apply=False, p=4.151788295496297e-06, threshold=(43, 43)),
        HorizontalFlip(always_apply=False, p=2.44894711000606e-08),
        VerticalFlip(always_apply=False, p=9.45963821614067e-09),
        ShiftScaleRotate(always_apply=False, p=5.451181222454858e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-72.05238342285156, -72.05238342285156), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0, shift_limit_x=(0.029665470123291016, 0.029665470123291016), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5107894438642515e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.6860225200653076, -0.6860225200653076), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.011435903074538123, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.379253387451172, 9.379253387451172), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.01619195376343363, max_holes=16, max_height=318, max_width=318, min_holes=16, min_height=318, min_width=318, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.7054840922355652, max_holes=16, max_height=37, max_width=37, min_holes=16, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.2665488148824905),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=4.844516487273719e-08, r_shift_limit=(-141, -141), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=9.467245653631013e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-10, -10), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.7235931935072305e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-2, -2)),
        RandomBrightnessContrast(always_apply=False, p=0.004257587294568488, brightness_limit=(0.7112568616867065, 0.7112568616867065), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.203767099768228e-05, brightness_limit=(0, 0), contrast_limit=(10.0, 10.0), brightness_by_max=True),
        Solarize(always_apply=False, p=4.4361824907638877e-11, threshold=(130, 130)),
        HorizontalFlip(always_apply=False, p=1.8687630733549407e-06),
        VerticalFlip(always_apply=False, p=0.14086440303620051),
        ShiftScaleRotate(always_apply=False, p=1.4725409729188002e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(169.9661865234375, 169.9661865234375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4996813157375722e-10, shift_limit_x=(0.29675984382629395, 0.29675984382629395), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00016017791034830114, shift_limit_x=(0, 0), shift_limit_y=(-0.3231474757194519, -0.3231474757194519), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.2871011269745005e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.218820571899414, 8.218820571899414), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.2933995609513435e-09, max_holes=16, max_height=28, max_width=28, min_holes=16, min_height=28, min_width=28, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.5536956684226895e-06, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8545864518678823),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.0588747876831194e-07, r_shift_limit=(106, 106), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=7.029499775352752e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-203, -203), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.850496829763169e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(97, 97)),
        RandomBrightnessContrast(always_apply=False, p=0.06701818753583222, brightness_limit=(-0.35999399423599243, -0.35999399423599243), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.48002191094337e-06, brightness_limit=(0, 0), contrast_limit=(5.093861103057861, 5.093861103057861), brightness_by_max=True),
        Solarize(always_apply=False, p=4.387667953012302e-06, threshold=(156, 156)),
        HorizontalFlip(always_apply=False, p=0.0002826829775680051),
        VerticalFlip(always_apply=False, p=4.537107480321627e-07),
        ShiftScaleRotate(always_apply=False, p=1.3229118608457276e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(116.95404052734375, 116.95404052734375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1316342324093075e-06, shift_limit_x=(0.12220799922943115, 0.12220799922943115), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.03586105767317749, shift_limit_x=(0, 0), shift_limit_y=(-0.38269054889678955, -0.38269054889678955), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0010150541437128346, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.517642021179199, 4.517642021179199), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.12649858236854516, max_holes=16, max_height=59, max_width=59, min_holes=16, min_height=59, min_width=59, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.014826117607491085, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7544398951846686),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.4629887872105885e-07, r_shift_limit=(-51, -51), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00010524178907218157, r_shift_limit=(-0.0, 0.0), g_shift_limit=(159, 159), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.009616987979654823, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-189, -189)),
        RandomBrightnessContrast(always_apply=False, p=1.7576903238521353e-05, brightness_limit=(0.892719030380249, 0.892719030380249), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.015366190407515301, brightness_limit=(0, 0), contrast_limit=(8.198083877563477, 8.198083877563477), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0007694446732550569, threshold=(87, 87)),
        HorizontalFlip(always_apply=False, p=1.9120671855081534e-08),
        VerticalFlip(always_apply=False, p=4.309648853072369e-10),
        ShiftScaleRotate(always_apply=False, p=0.07917191571713289, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-94.24112701416016, -94.24112701416016), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6211146409599638e-06, shift_limit_x=(0.5317703485488892, 0.5317703485488892), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.102500830407796e-08, shift_limit_x=(0, 0), shift_limit_y=(0.6168783903121948, 0.6168783903121948), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.9685792898463165e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.471087455749512, 5.471087455749512), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.05477549880743027, max_holes=16, max_height=465, max_width=465, min_holes=16, min_height=465, min_width=465, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.006175543254829108, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8339956738984173),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.0128066903029641e-10, r_shift_limit=(255, 255), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.1689114714460425e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(22, 22), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.5372784282103803e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(83, 83)),
        RandomBrightnessContrast(always_apply=False, p=2.1950315007370814e-08, brightness_limit=(-0.69361412525177, -0.69361412525177), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.28196437468499e-09, brightness_limit=(0, 0), contrast_limit=(10.0, 10.0), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0002035863024475986, threshold=(137, 137)),
        HorizontalFlip(always_apply=False, p=6.782377283843439e-07),
        VerticalFlip(always_apply=False, p=0.017360624812706393),
        ShiftScaleRotate(always_apply=False, p=1.4842902234339961e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-170.3380889892578, -170.3380889892578), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0007085032437635745, shift_limit_x=(-0.8736046552658081, -0.8736046552658081), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.002772481772155694, shift_limit_x=(0, 0), shift_limit_y=(-0.4565311074256897, -0.4565311074256897), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.930798204989074, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.389795780181885, 4.389795780181885), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.1807473852300514e-05, max_holes=16, max_height=44, max_width=44, min_holes=16, min_height=44, min_width=44, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.9933201365786675e-05, max_holes=11, max_height=37, max_width=37, min_holes=11, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.04811096224169409),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=9.178587607830396e-10, r_shift_limit=(-145, -145), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.48376397413572647, r_shift_limit=(-0.0, 0.0), g_shift_limit=(14, 14), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.68597529513785e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-49, -49)),
        RandomBrightnessContrast(always_apply=False, p=0.03064258476328341, brightness_limit=(0.31123828887939453, 0.31123828887939453), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.01251209822441024, brightness_limit=(0, 0), contrast_limit=(8.51723861694336, 8.51723861694336), brightness_by_max=True),
        Solarize(always_apply=False, p=6.421835043007423e-09, threshold=(27, 27)),
        HorizontalFlip(always_apply=False, p=0.015864847347747846),
        VerticalFlip(always_apply=False, p=0.000183954580610865),
        ShiftScaleRotate(always_apply=False, p=0.00010698844833581696, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(77.22744750976562, 77.22744750976562), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00019390471570113026, shift_limit_x=(-0.25135093927383423, -0.25135093927383423), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.764474332525787e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.25923460721969604, -0.25923460721969604), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.162147763975803e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.8936543464660645, 1.8936543464660645), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.00045100991390090855, max_holes=16, max_height=575, max_width=575, min_holes=16, min_height=575, min_width=575, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=5.6656676404772885e-05, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4561968974100097),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.04208346054213408, r_shift_limit=(22, 22), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.202115160709778e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(120, 120), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.002732832223127493, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-134, -134)),
        RandomBrightnessContrast(always_apply=False, p=8.671189630325845e-07, brightness_limit=(-0.12754422426223755, -0.12754422426223755), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=4.601435637334528e-09, brightness_limit=(0, 0), contrast_limit=(7.515273571014404, 7.515273571014404), brightness_by_max=True),
        Solarize(always_apply=False, p=0.005876845325434477, threshold=(193, 193)),
        HorizontalFlip(always_apply=False, p=3.7991273830980293e-08),
        VerticalFlip(always_apply=False, p=0.0031344857686483873),
        ShiftScaleRotate(always_apply=False, p=3.0077437147953927e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(105.30636596679688, 105.30636596679688), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.8441274163183835e-06, shift_limit_x=(-0.28599482774734497, -0.28599482774734497), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=7.205140707127796e-09, shift_limit_x=(0, 0), shift_limit_y=(0.32540059089660645, 0.32540059089660645), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.3130515903111007e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.513957977294922, 4.513957977294922), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.5928060060654677e-07, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.1537606048788387, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7924050329089098),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=5.28538696393003e-06, r_shift_limit=(-105, -105), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.000777478455248927, r_shift_limit=(-0.0, 0.0), g_shift_limit=(41, 41), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.43077701029641347, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-51, -51)),
        RandomBrightnessContrast(always_apply=False, p=0.014163535623541779, brightness_limit=(0.3858964443206787, 0.3858964443206787), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.012864167112927538, brightness_limit=(0, 0), contrast_limit=(5.41510009765625, 5.41510009765625), brightness_by_max=True),
        Solarize(always_apply=False, p=1.212845325284573e-09, threshold=(23, 23)),
        HorizontalFlip(always_apply=False, p=4.2235140736280165e-06),
        VerticalFlip(always_apply=False, p=4.161801772724018e-10),
        ShiftScaleRotate(always_apply=False, p=8.24325668231235e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-156.16995239257812, -156.16995239257812), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.211628707975953e-10, shift_limit_x=(0.39498960971832275, 0.39498960971832275), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.686317280585018e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.4194979667663574, -0.4194979667663574), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.663655128373892e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(5.431559085845947, 5.431559085845947), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.016471542940005746, max_holes=16, max_height=214, max_width=214, min_holes=16, min_height=214, min_width=214, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.519152869557729e-06, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5248502695722582),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.0005720114553202649, r_shift_limit=(-91, -91), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.8967558369867005e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-203, -203), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00018879276211530502, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-254, -254)),
        RandomBrightnessContrast(always_apply=False, p=5.754940957558574e-06, brightness_limit=(0.6562933921813965, 0.6562933921813965), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.2924996100766464, brightness_limit=(0, 0), contrast_limit=(1.0419772863388062, 1.0419772863388062), brightness_by_max=True),
        Solarize(always_apply=False, p=0.00257841534384691, threshold=(76, 76)),
        HorizontalFlip(always_apply=False, p=0.0002649537632057868),
        VerticalFlip(always_apply=False, p=9.168760754682493e-05),
        ShiftScaleRotate(always_apply=False, p=5.021832659096768e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-56.50226593017578, -56.50226593017578), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0007438802160761558, shift_limit_x=(-0.3146411180496216, -0.3146411180496216), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00012200240955748207, shift_limit_x=(0, 0), shift_limit_y=(-0.4676690101623535, -0.4676690101623535), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6002065609464192e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(4.845980644226074, 4.845980644226074), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=4.524227826193132e-06, max_holes=16, max_height=507, max_width=507, min_holes=16, min_height=507, min_width=507, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0033855514670271813, max_holes=4, max_height=37, max_width=37, min_holes=4, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6995406843724887),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.12687501868271056, r_shift_limit=(56, 56), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.08887224087863821, r_shift_limit=(-0.0, 0.0), g_shift_limit=(195, 195), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.14469047700207316, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-73, -73)),
        RandomBrightnessContrast(always_apply=False, p=0.006412636686883744, brightness_limit=(0.195855975151062, 0.195855975151062), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=9.69191180353663e-07, brightness_limit=(0, 0), contrast_limit=(2.7091622352600098, 2.7091622352600098), brightness_by_max=True),
        Solarize(always_apply=False, p=1.98967435457968e-09, threshold=(153, 153)),
        HorizontalFlip(always_apply=False, p=2.323690737470217e-06),
        VerticalFlip(always_apply=False, p=3.2168824370374e-06),
        ShiftScaleRotate(always_apply=False, p=0.00011330152406461257, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-104.38568115234375, -104.38568115234375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.417933766144848e-06, shift_limit_x=(0.6128842830657959, 0.6128842830657959), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.7127355749583544e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.9268943667411804, -0.9268943667411804), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=9.42643608496769e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.48878812789917, 2.48878812789917), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.724158477313289e-07, max_holes=16, max_height=429, max_width=429, min_holes=16, min_height=429, min_width=429, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00027625531147008886, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6326978395121232),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.4254560233836514e-09, r_shift_limit=(-108, -108), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.20853974918960816, r_shift_limit=(-0.0, 0.0), g_shift_limit=(26, 26), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=4.6373037452807736e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(228, 228)),
        RandomBrightnessContrast(always_apply=False, p=0.008714091038831251, brightness_limit=(0.17507719993591309, 0.17507719993591309), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=1.9015408344760613e-05, brightness_limit=(0, 0), contrast_limit=(3.3387179374694824, 3.3387179374694824), brightness_by_max=True),
        Solarize(always_apply=False, p=0.008234640458976372, threshold=(196, 196)),
        HorizontalFlip(always_apply=False, p=0.0002257220432224094),
        VerticalFlip(always_apply=False, p=3.475813000549123e-09),
        ShiftScaleRotate(always_apply=False, p=5.222561995568581e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-80.68161010742188, -80.68161010742188), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00025737336052123264, shift_limit_x=(-0.21088087558746338, -0.21088087558746338), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00382519158987793, shift_limit_x=(0, 0), shift_limit_y=(-0.8267574310302734, -0.8267574310302734), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002027963808636738, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.4771153926849365, 3.4771153926849365), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.013592370318343372, max_holes=16, max_height=288, max_width=288, min_holes=16, min_height=288, min_width=288, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.16263652208900048, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.5937524753258476),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.7023340088599107e-11, r_shift_limit=(-42, -42), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.22423270170443e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(239, 239), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.421915694966807e-12, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(252, 252)),
        RandomBrightnessContrast(always_apply=False, p=1.359826986858542e-07, brightness_limit=(-0.6471456289291382, -0.6471456289291382), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.4867472047285083e-10, brightness_limit=(0, 0), contrast_limit=(8.7691011428833, 8.7691011428833), brightness_by_max=True),
        Solarize(always_apply=False, p=4.6355752627944e-05, threshold=(99, 99)),
        HorizontalFlip(always_apply=False, p=3.0012928490531625e-10),
        VerticalFlip(always_apply=False, p=4.6232918804600946e-10),
        ShiftScaleRotate(always_apply=False, p=5.067785079594406e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(8.30401611328125, 8.30401611328125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.9387735103402974, shift_limit_x=(0.33291375637054443, 0.33291375637054443), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.3299046849774097e-05, shift_limit_x=(0, 0), shift_limit_y=(0.5018961429595947, 0.5018961429595947), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.250912297121848e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.001877784729004, 2.001877784729004), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.864654346596524e-11, max_holes=16, max_height=251, max_width=251, min_holes=16, min_height=251, min_width=251, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.7707625779094583e-06, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.061159670923217724),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.00182980288167231, r_shift_limit=(143, 143), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=6.507159618852204e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-236, -236), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0010588558462871767, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(90, 90)),
        RandomBrightnessContrast(always_apply=False, p=4.8033568067161255e-09, brightness_limit=(-0.48280543088912964, -0.48280543088912964), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.00023110923725687751, brightness_limit=(0, 0), contrast_limit=(1.0668199062347412, 1.0668199062347412), brightness_by_max=True),
        Solarize(always_apply=False, p=1.7654523675900262e-09, threshold=(69, 69)),
        HorizontalFlip(always_apply=False, p=4.753401841737339e-11),
        VerticalFlip(always_apply=False, p=0.32837593712116764),
        ShiftScaleRotate(always_apply=False, p=1.4509514911503298e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(76.78073120117188, 76.78073120117188), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8419811631267499e-06, shift_limit_x=(-0.13309258222579956, -0.13309258222579956), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00033711317765596285, shift_limit_x=(0, 0), shift_limit_y=(0.3302654027938843, 0.3302654027938843), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.456603519878578e-10, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.87020492553711, 8.87020492553711), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=7.72436096113692e-09, max_holes=16, max_height=412, max_width=412, min_holes=16, min_height=412, min_width=412, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0032103758967858, max_holes=5, max_height=37, max_width=37, min_holes=5, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6648898628659432),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=2.724304795476655e-07, r_shift_limit=(-173, -173), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.3667539071422041, r_shift_limit=(-0.0, 0.0), g_shift_limit=(70, 70), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0025965098134829723, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-125, -125)),
        RandomBrightnessContrast(always_apply=False, p=0.30763358451839906, brightness_limit=(0.08194482326507568, 0.08194482326507568), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=4.338151568146313e-05, brightness_limit=(0, 0), contrast_limit=(2.3448898792266846, 2.3448898792266846), brightness_by_max=True),
        Solarize(always_apply=False, p=0.012737859669968432, threshold=(110, 110)),
        HorizontalFlip(always_apply=False, p=0.00035949131562669837),
        VerticalFlip(always_apply=False, p=5.067281364678021e-07),
        ShiftScaleRotate(always_apply=False, p=0.0008366992724687106, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-133.16128540039062, -133.16128540039062), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.00708072707210694, shift_limit_x=(0.8080986738204956, 0.8080986738204956), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=4.704222386774591e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.8433345556259155, -0.8433345556259155), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5205399721184498e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.202625274658203, 2.202625274658203), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.1440931896391824e-08, max_holes=16, max_height=468, max_width=468, min_holes=16, min_height=468, min_width=468, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=2.691965616566502e-05, max_holes=8, max_height=37, max_width=37, min_holes=8, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.30192860418015344),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.15551497087767174, r_shift_limit=(-17, -17), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.005089945719331496, r_shift_limit=(-0.0, 0.0), g_shift_limit=(139, 139), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.6005981353184513e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-201, -201)),
        RandomBrightnessContrast(always_apply=False, p=1.02560283172744e-10, brightness_limit=(-0.8341546058654785, -0.8341546058654785), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.12706950414567775, brightness_limit=(0, 0), contrast_limit=(1.5257568359375, 1.5257568359375), brightness_by_max=True),
        Solarize(always_apply=False, p=0.0, threshold=(119, 119)),
        HorizontalFlip(always_apply=False, p=0.0016978951255401664),
        VerticalFlip(always_apply=False, p=0.0002882703392122956),
        ShiftScaleRotate(always_apply=False, p=0.03187994747570233, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(63.905670166015625, 63.905670166015625), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.12588398967009e-05, shift_limit_x=(-0.4866291880607605, -0.4866291880607605), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.2140581726195806e-06, shift_limit_x=(0, 0), shift_limit_y=(0.038243770599365234, 0.038243770599365234), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.4304068827186711e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.971421241760254, 6.971421241760254), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.05910117380258617, max_holes=16, max_height=557, max_width=557, min_holes=16, min_height=557, min_width=557, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=3.228226016395788e-05, max_holes=14, max_height=37, max_width=37, min_holes=14, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6192812272033041),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.4036681793261214e-09, r_shift_limit=(156, 156), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=3.504851985547523e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(170, 170), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.6559259055238836e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(101, 101)),
        RandomBrightnessContrast(always_apply=False, p=3.2788084416131513e-09, brightness_limit=(0.8500238656997681, 0.8500238656997681), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=5.007154361145675e-11, brightness_limit=(0, 0), contrast_limit=(6.084325790405273, 6.084325790405273), brightness_by_max=True),
        Solarize(always_apply=False, p=4.318469506813815e-11, threshold=(143, 143)),
        HorizontalFlip(always_apply=False, p=2.0673086949508115e-08),
        VerticalFlip(always_apply=False, p=2.5672242175685215e-11),
        ShiftScaleRotate(always_apply=False, p=8.628158658522502e-12, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-7.887725830078125, -7.887725830078125), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.641654417230934e-11, shift_limit_x=(-0.4792194366455078, -0.4792194366455078), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.6081238031637071, shift_limit_x=(0, 0), shift_limit_y=(-0.8326767086982727, -0.8326767086982727), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8587708665245115e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.5981413125991821, 0.5981413125991821), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=3.9768616587696976e-08, max_holes=16, max_height=220, max_width=220, min_holes=16, min_height=220, min_width=220, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=6.895952557091728e-08, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.39187417488800325),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.4966682535574329e-05, r_shift_limit=(-87, -87), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.9371092703058666e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-127, -127), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00040031490167158346, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(84, 84)),
        RandomBrightnessContrast(always_apply=False, p=8.076503658601587e-08, brightness_limit=(-0.5451623797416687, -0.5451623797416687), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.588750789694366e-08, brightness_limit=(0, 0), contrast_limit=(2.2007875442504883, 2.2007875442504883), brightness_by_max=True),
        Solarize(always_apply=False, p=0.20125941927196322, threshold=(210, 210)),
        HorizontalFlip(always_apply=False, p=3.5185494914769875e-07),
        VerticalFlip(always_apply=False, p=4.0287107865525145e-08),
        ShiftScaleRotate(always_apply=False, p=3.892218077993566e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-180.0, -180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.06150768265689e-07, shift_limit_x=(0.2815864086151123, 0.2815864086151123), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.7894797545625531e-06, shift_limit_x=(0, 0), shift_limit_y=(0.0065305233001708984, 0.0065305233001708984), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.420255896698798e-11, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.565784931182861, 6.565784931182861), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.320909874647451e-09, max_holes=16, max_height=600, max_width=600, min_holes=16, min_height=600, min_width=600, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.006713890946127954, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.7916043997982942),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.00036935397941849357, r_shift_limit=(24, 24), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.430688920771954e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-19, -19), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.923676065986298e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-133, -133)),
        RandomBrightnessContrast(always_apply=False, p=7.94522295143295e-08, brightness_limit=(0.6034146547317505, 0.6034146547317505), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.0264684663821715e-09, brightness_limit=(0, 0), contrast_limit=(3.693113088607788, 3.693113088607788), brightness_by_max=True),
        Solarize(always_apply=False, p=3.621688237467381e-06, threshold=(146, 146)),
        HorizontalFlip(always_apply=False, p=5.341463270432502e-06),
        VerticalFlip(always_apply=False, p=3.558602127631816e-05),
        ShiftScaleRotate(always_apply=False, p=0.0005406336321379865, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(81.35061645507812, 81.35061645507812), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.003603833665662748, shift_limit_x=(-0.43283379077911377, -0.43283379077911377), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.8984759612124963e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.34928959608078003, -0.34928959608078003), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.012144270206624341, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(6.930058002471924, 6.930058002471924), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.3685160907793127e-06, max_holes=16, max_height=415, max_width=415, min_holes=16, min_height=415, min_width=415, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.3292201150784422, max_holes=1, max_height=37, max_width=37, min_holes=1, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.6540747849049204),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=2.2700208599202222e-09, r_shift_limit=(52, 52), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.0, r_shift_limit=(-0.0, 0.0), g_shift_limit=(214, 214), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.1166104139176555e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(255, 255)),
        RandomBrightnessContrast(always_apply=False, p=3.7883859701858154e-05, brightness_limit=(0.5573800802230835, 0.5573800802230835), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.009379331418939707, brightness_limit=(0, 0), contrast_limit=(4.455543518066406, 4.455543518066406), brightness_by_max=True),
        Solarize(always_apply=False, p=3.6295102865604046e-06, threshold=(33, 33)),
        HorizontalFlip(always_apply=False, p=2.141674039712173e-08),
        VerticalFlip(always_apply=False, p=0.001514942401438131),
        ShiftScaleRotate(always_apply=False, p=2.2973709009704464e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-0.0597991943359375, -0.0597991943359375), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.1894434927589004e-07, shift_limit_x=(-0.0010442137718200684, -0.0010442137718200684), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=8.389991120668573e-08, shift_limit_x=(0, 0), shift_limit_y=(-0.6278845071792603, -0.6278845071792603), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0005350085746435793, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(3.1893978118896484, 3.1893978118896484), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.001325696914516325, max_holes=16, max_height=478, max_width=478, min_holes=16, min_height=478, min_width=478, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.9740861654281616, max_holes=9, max_height=37, max_width=37, min_holes=9, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.013117011947309187),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=1.182307683216254e-08, r_shift_limit=(-77, -77), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=2.977763796139192e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(135, 135), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.004329445771872997, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(69, 69)),
        RandomBrightnessContrast(always_apply=False, p=3.353001497231442e-09, brightness_limit=(-0.4597126841545105, -0.4597126841545105), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0005034331898511185, brightness_limit=(0, 0), contrast_limit=(6.878686904907227, 6.878686904907227), brightness_by_max=True),
        Solarize(always_apply=False, p=4.809743046934499e-08, threshold=(136, 136)),
        HorizontalFlip(always_apply=False, p=2.7930195730450858e-08),
        VerticalFlip(always_apply=False, p=5.273642023080856e-11),
        ShiftScaleRotate(always_apply=False, p=0.743118939152172, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(180.0, 180.0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.2407201944978784e-06, shift_limit_x=(-0.06646114587783813, -0.06646114587783813), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=6.542814166956555e-10, shift_limit_x=(0, 0), shift_limit_y=(-0.5371958017349243, -0.5371958017349243), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.004948356467359555, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.0266027450561523, 1.0266027450561523), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.444471956083471e-07, max_holes=16, max_height=162, max_width=162, min_holes=16, min_height=162, min_width=162, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0001411781889787255, max_holes=14, max_height=37, max_width=37, min_holes=14, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.24695674037401516),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=3.5516465996909656e-05, r_shift_limit=(140, 140), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.70717746583095e-09, r_shift_limit=(-0.0, 0.0), g_shift_limit=(146, 146), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.00017661027111650577, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-255, -255)),
        RandomBrightnessContrast(always_apply=False, p=0.00021025940140370278, brightness_limit=(0.6287746429443359, 0.6287746429443359), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.80535452040448e-05, brightness_limit=(0, 0), contrast_limit=(4.81199836730957, 4.81199836730957), brightness_by_max=True),
        Solarize(always_apply=False, p=8.278054628739538e-07, threshold=(107, 107)),
        HorizontalFlip(always_apply=False, p=3.589885330950869e-05),
        VerticalFlip(always_apply=False, p=1.0413429165456514e-10),
        ShiftScaleRotate(always_apply=False, p=2.53537738642774e-09, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-87.12435150146484, -87.12435150146484), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0001251344521111782, shift_limit_x=(0.5449309349060059, 0.5449309349060059), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.1369632982025687e-06, shift_limit_x=(0, 0), shift_limit_y=(-0.518928050994873, -0.518928050994873), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.16865470306779073, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(2.9442694187164307, 2.9442694187164307), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=1.1141237763382587e-08, max_holes=16, max_height=463, max_width=463, min_holes=16, min_height=463, min_width=463, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=5.567674215516399e-10, max_holes=15, max_height=37, max_width=37, min_holes=15, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.8307218391296121),
      ], p=1.0),
    ], p=0.03333333333333333),
    Sequential([
      OneOf([
        RGBShift(always_apply=False, p=0.2815288928057207, r_shift_limit=(-44, -44), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.299337777137297e-07, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-80, -80), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.1733798899310873e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-59, -59)),
        RandomBrightnessContrast(always_apply=False, p=1.882532481050183e-05, brightness_limit=(-0.525475800037384, -0.525475800037384), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=3.7642419976386163e-07, brightness_limit=(0, 0), contrast_limit=(0.0, 0.0), brightness_by_max=True),
        Solarize(always_apply=False, p=1.2409133006967103e-05, threshold=(158, 158)),
        HorizontalFlip(always_apply=False, p=8.659215874038036e-09),
        VerticalFlip(always_apply=False, p=9.443712501569675e-05),
        ShiftScaleRotate(always_apply=False, p=1.4467613984457107e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-109.3427963256836, -109.3427963256836), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.7173482463883075e-10, shift_limit_x=(-0.4787141680717468, -0.4787141680717468), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.832121309142354e-08, shift_limit_x=(0, 0), shift_limit_y=(0.22256362438201904, 0.22256362438201904), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.000275185229921069, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(1.9806594848632812, 1.9806594848632812), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.2623438328922667, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.0007639160612598062, max_holes=7, max_height=37, max_width=37, min_holes=7, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.4549606297618274),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=4.704940024676143e-09, r_shift_limit=(-232, -232), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.2724397434286323e-06, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-167, -167), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=1.2586187575494719e-05, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(242, 242)),
        RandomBrightnessContrast(always_apply=False, p=7.57522675701866e-07, brightness_limit=(-0.4795342683792114, -0.4795342683792114), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=2.8369287769190732e-05, brightness_limit=(0, 0), contrast_limit=(7.398906230926514, 7.398906230926514), brightness_by_max=True),
        Solarize(always_apply=False, p=2.0769751876642615e-06, threshold=(88, 88)),
        HorizontalFlip(always_apply=False, p=0.5943641163874993),
        VerticalFlip(always_apply=False, p=1.1322496009384103e-07),
        ShiftScaleRotate(always_apply=False, p=7.140054515973478e-08, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-122.79579162597656, -122.79579162597656), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002050620848000964, shift_limit_x=(-0.2110951542854309, -0.2110951542854309), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.681226368010791e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.35223501920700073, -0.35223501920700073), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=2.0601477071623825e-06, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(8.146245956420898, 8.146245956420898), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=5.41070861969405e-05, max_holes=16, max_height=391, max_width=391, min_holes=16, min_height=391, min_width=391, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00046037692759737237, max_holes=10, max_height=37, max_width=37, min_holes=10, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.40483221335912223),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=7.64686359040382e-06, r_shift_limit=(-124, -124), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.736026927629437e-10, r_shift_limit=(-0.0, 0.0), g_shift_limit=(154, 154), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.009710067454928417, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-166, -166)),
        RandomBrightnessContrast(always_apply=False, p=0.00022474983790892258, brightness_limit=(1.0, 1.0), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=6.440695059632579e-09, brightness_limit=(0, 0), contrast_limit=(8.905611038208008, 8.905611038208008), brightness_by_max=True),
        Solarize(always_apply=False, p=9.19526108521185e-09, threshold=(176, 176)),
        HorizontalFlip(always_apply=False, p=3.753899556773366e-06),
        VerticalFlip(always_apply=False, p=3.753425136750763e-05),
        ShiftScaleRotate(always_apply=False, p=5.95469591936253e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(47.79496765136719, 47.79496765136719), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.465874394373631e-09, shift_limit_x=(0.4433107376098633, 0.4433107376098633), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.6270701975111372e-05, shift_limit_x=(0, 0), shift_limit_y=(-0.4831269383430481, -0.4831269383430481), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=1.5415538498924055e-07, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(10.0, 10.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=0.5263192195493147, max_holes=16, max_height=1, max_width=1, min_holes=16, min_height=1, min_width=1, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.00010257053970863517, max_holes=6, max_height=37, max_width=37, min_holes=6, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.46351846811163766),
      ], p=1.0),
      OneOf([
        RGBShift(always_apply=False, p=0.05736473471629111, r_shift_limit=(40, 40), g_shift_limit=(-0.0, 0.0), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=5.316593145350139e-08, r_shift_limit=(-0.0, 0.0), g_shift_limit=(84, 84), b_shift_limit=(-0.0, 0.0)),
        RGBShift(always_apply=False, p=0.04781665650043454, r_shift_limit=(-0.0, 0.0), g_shift_limit=(-0.0, 0.0), b_shift_limit=(69, 69)),
        RandomBrightnessContrast(always_apply=False, p=1.2307401432907876e-07, brightness_limit=(-0.1318928599357605, -0.1318928599357605), contrast_limit=(0, 0), brightness_by_max=True),
        RandomBrightnessContrast(always_apply=False, p=0.0002098050558479031, brightness_limit=(0, 0), contrast_limit=(4.7193169593811035, 4.7193169593811035), brightness_by_max=True),
        Solarize(always_apply=False, p=2.1737192631567123e-07, threshold=(231, 231)),
        HorizontalFlip(always_apply=False, p=2.836987001935596e-08),
        VerticalFlip(always_apply=False, p=0.0011765817953564017),
        ShiftScaleRotate(always_apply=False, p=1.6598079538102388e-05, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(-116.39137268066406, -116.39137268066406), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=5.151344676006253e-05, shift_limit_x=(0.318332314491272, 0.318332314491272), shift_limit_y=(0, 0), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=3.4911127482056666e-09, shift_limit_x=(0, 0), shift_limit_y=(-0.429926335811615, -0.429926335811615), scale_limit=(0.0, 0.0), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        ShiftScaleRotate(always_apply=False, p=0.0002713532871806649, shift_limit_x=(0, 0), shift_limit_y=(0, 0), scale_limit=(9.99999993922529e-09, 9.99999993922529e-09), rotate_limit=(0, 0), interpolation=1, border_mode=4, value=None, mask_value=None),
        CoarseDropout(always_apply=False, p=2.98852580623535e-06, max_holes=16, max_height=51, max_width=51, min_holes=16, min_height=51, min_width=51, fill_value=0, mask_fill_value=None),
        CoarseDropout(always_apply=False, p=0.5516236076832115, max_holes=13, max_height=37, max_width=37, min_holes=13, min_height=37, min_width=37, fill_value=0, mask_fill_value=None),
        NoOp(always_apply=False, p=0.34146573543671854),
      ], p=1.0),
    ], p=0.03333333333333333),
  ], p=1),
  Normalize(always_apply=True, p=1.0, mean=[0.48500001430511475, 0.4560000002384186, 0.4059999883174896], std=[0.2290000021457672, 0.2240000069141388, 0.22499999403953552], max_pixel_value=255),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=True),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            #Resize(CFG.size, CFG.size),
            RandomResizedCrop(CFG.size, CFG.size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
        #return A.load("../input/autoalbument/latest.json")
        #return train_aug_policy

    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
train_dataset = TrainDataset(train, transform=get_transforms(data='train'))

for i in range(1):
    image, label = train_dataset[i]
    plt.imshow(image[0])
    plt.title(f'label: {label}')
    plt.show() 

### RayTune Dataset class

In [ ]:
# ===================
# Only for Raytune
# ===================

def load_data(data_dir):   
    folds = data_dir
    
    trn_idx = folds[folds['fold'] != 0].index
    val_idx = folds[folds['fold'] == 0].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)

    train_dataset = TrainDataset(train_folds, 
                                 transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, 
                                 transform=get_transforms(data='valid'))
    
    return train_dataset, valid_dataset

# MODEL

In [ ]:
# ====================================================
# MODEL
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = CustomResNext(model_name=CFG.model_name, pretrained=False)
train_dataset = TrainDataset(train, transform=get_transforms(data='train'))
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True,
                          num_workers=4, pin_memory=True, drop_last=True)

for image, label in train_loader:
    output = model(image)
    print(output)
    break

# Helper functions

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    iters = len(train_loader)
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(images)
        loss = criterion(y_preds, labels)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step(epoch + step / iters)
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.softmax(1).to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state['model'])
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

# Train loop

In [ ]:
# ====================================================
# Train loop
# ====================================================
#def train_loop(config, checkpoint_dir=None, data_dir=None):
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    if not CFG.tune:
        trn_idx = folds[folds['fold'] != fold].index
        val_idx = folds[folds['fold'] == fold].index

        train_folds = folds.loc[trn_idx].reset_index(drop=True)
        valid_folds = folds.loc[val_idx].reset_index(drop=True)

        train_dataset = TrainDataset(train_folds, 
                                     transform=get_transforms(data='train'))
        valid_dataset = TrainDataset(valid_folds, 
                                     transform=get_transforms(data='valid'))        
        train_loader = DataLoader(train_dataset, 
                                  batch_size=CFG.batch_size, 
                                  shuffle=True, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
        valid_loader = DataLoader(valid_dataset, 
                                  batch_size=CFG.batch_size, 
                                  shuffle=False, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=False)      
    else:
        train_dataset, valid_dataset = load_data(data_dir)
        
        train_loader = DataLoader(train_dataset, 
                                  batch_size=tuneCFG["batch_size"], 
                                  shuffle=True, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
        valid_loader = DataLoader(valid_dataset, 
                                  batch_size=tuneCFG["batch_size"], 
                                  shuffle=False, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=False)   

        
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomResNext(CFG.model_name, pretrained=True)
    model.to(device)

    if CFG.tune:
        optimizer = Adam(model.parameters(), lr=tuneCFG["lr"], weight_decay=tuneCFG["weight_decay"], amsgrad=False)
    else:
        optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    
    scheduler = get_scheduler(optimizer)

    
    # ====================================================
    # RayTune checkpoints
    # ====================================================
    if CFG.tune:
        if checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(checkpoint_dir, "checkpoint"))
            model.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    
    # ====================================================
    # apex
    # ====================================================
    if CFG.apex:
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.CrossEntropyLoss()

    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds[CFG.target_col].values
        
        #if isinstance(scheduler, ReduceLROnPlateau):
        #    scheduler.step(avg_val_loss)
        #elif isinstance(scheduler, CosineAnnealingLR):
        #    scheduler.step()
        #elif isinstance(scheduler, CosineAnnealingWarmRestarts):
        #    scheduler.step()

        # scoring
        score = get_score(valid_labels, preds.argmax(1))

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Accuracy: {score}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
        
        # === RAYTUNE ===
        if CFG.tune:
            with tune.checkpoint_dir(epoch) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                torch.save((model.state_dict(), optimizer.state_dict()), path)

            #tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
            tune.report(loss=avg_val_loss, accuracy=score)

    
    check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    valid_folds[[str(c) for c in range(5)]] = check_point['preds']
    valid_folds['preds'] = check_point['preds'].argmax(1)

    if not CFG.tune:
        return valid_folds

## RayTune training function

In [ ]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [ ]:
"""

def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

"""

In [ ]:
# ====================================================
# main
# ====================================================
def main(num_samples=10, max_num_epochs=5, gpus_per_trial=1):

    """
    Prepare: 1.train  2.test  3.submission  4.folds
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.5f}')
    
    if CFG.train:
        if CFG.tune:
            #data_dir = os.path.abspath("./data")
            #checkpoint_dir = os.path("./raytune")

            #load_data(data_dir)
            config = {
                "lr": tune.loguniform(1e-4, 1e-1),
                "batch_size": tune.choice([2, 4, 8, 16]),
                "weight_decay": tune.loguniform(1e-8, 1e-4)
            }
            scheduler = ASHAScheduler(
                metric="loss",
                mode="min",
                max_t=max_num_epochs,
                grace_period=1,
                reduction_factor=2)
            reporter = CLIReporter(
                # parameter_columns=["l1", "l2", "lr", "batch_size"],
                metric_columns=["loss", "accuracy", "training_iteration"])
            
            result = tune.run(
                partial(train_loop, checkpoint_dir=checkpoint_dir, data_dir=folds),
                resources_per_trial={"gpu": gpus_per_trial},
                config=config,
                num_samples=num_samples,
                scheduler=scheduler,
                progress_reporter=reporter)
            
            best_trial = result.get_best_trial("loss", "min", "last")
            print("Best trial config: {}".format(best_trial.config))
            print("Best trial final validation loss: {}".format(
                best_trial.last_result["loss"]))
            print("Best trial final validation accuracy: {}".format(
                best_trial.last_result["accuracy"]))

            #best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
            #device = "cpu"
            #if torch.cuda.is_available():
            #    device = "cuda:0"
            #    if gpus_per_trial > 1:
            #        best_trained_model = nn.DataParallel(best_trained_model)
            #best_trained_model.to(device)

            #best_checkpoint_dir = best_trial.checkpoint.value
            #model_state, optimizer_state = torch.load(os.path.join(
            #    best_checkpoint_dir, "checkpoint"))
            #best_trained_model.load_state_dict(model_state)

            #test_acc = test_accuracy(best_trained_model, device)
            #print("Best trial test set accuracy: {}".format(test_acc))'
            
        else:
            # train 
            oof_df = pd.DataFrame()
            for fold in range(CFG.n_fold):
                if fold in CFG.trn_fold:
                    _oof_df = train_loop(folds, fold)
                    oof_df = pd.concat([oof_df, _oof_df])
                    LOGGER.info(f"========== fold: {fold} result ==========")
                    get_result(_oof_df)
            # CV result
            LOGGER.info(f"========== CV ==========")
            get_result(oof_df)
            # save result
            oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)
    
    if CFG.inference:
        # inference
        model = CustomResNext(CFG.model_name, pretrained=False)
        states = [torch.load(INPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
        test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                                 num_workers=CFG.num_workers, pin_memory=True)
        predictions = inference(model, states, test_loader, device)
        # submission
        test['label'] = predictions.argmax(1)
        test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)

In [ ]:
if __name__ == '__main__':
    main()